<a href="https://colab.research.google.com/github/brzenio/-TEST-Lubimyczytac-webscrap/blob/main/TODO_23_10_Multiscrap_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TODO
1. Eksport danych do JSONa
2. Sprawdzanie czy dana książka została zaimportowana, a jeśli nie, to wypisać która
3. Import z Goodreads dla angielskich książek
4. Eksport 2 tabel - jedna szczegółowa, tak jak teraz, a druga w wersji skróconej, aby się wyświetlała bez problemu na małych ekranach. Czyli Okładka, tytul, autor, ocena na Lubimyczytać, liczba stron i czas czytania (??? Dyskusyjne ostatnie)
5. Uporządkować kod, żeby był czytelniejszy, dodać komentarze, usunąc niepotrzebny kod
6. Koncepcja strony: 1. Dodać zależnośc wyświetlania strony od tego czy to jest telefon czy komputer. Na stronie głównej wyświetlić logo, prostokąt z okładkami książek?, i pod spodem dać okienko wyszukiwarki w kolorze białym. Generalnie skopiować to jak wygląda strona google. Założyć stronę Nowości, która pokazuje historię dodanych rzeczy (tu utworzyć funcję w Pyhonie, która appenduje nowe książki w zadanej linii kodu i dodaje datę, czyli maksymalnie zauktomatyzować cały ten proces. "


# Import

In [ ]:
import json
import requests
import time
import bs4
import pandas as pd
from datetime import datetime

In [ ]:
book_columns = ['image', 'book_title','name', 'datePublished', 'numberOfPages', 'ratingValue', 'ratingCount', 'category', 
       'publisher', 'series',
       'readingTime', 'tags', 'originalTitle', 'firstPolishPublish', 'originPublish',
        'isbn', 'inLanguage', 'url']

df = pd.DataFrame(columns = book_columns)
df

,image,book_title,name,datePublished,numberOfPages,ratingValue,ratingCount,category,publisher,series,readingTime,tags,originalTitle,firstPolishPublish,originPublish,isbn,inLanguage,url


In [ ]:
#url = 'https://lubimyczytac.pl/ksiazka/4890328/turbopatriotyzm'
books = []
book = {}
df3 = pd.DataFrame()
start_time = datetime.now()
print(start_time)
for el in book_urls:
  it = el
  url = book_urls[el]
  result = requests.get(url)
  soup = bs4.BeautifulSoup(result.text, "html.parser")

  data = json.loads(soup.find('script', type='application/ld+json').text)
  
  data2 = {}
  data2.update(data)
  data2['book_title'] = data2['name']
  agr_rating = 'aggregateRating'
  if agr_rating not in data:
    continue
  else:
      data2.update(data['aggregateRating'])
  data2.update(data['author'])
  to_del = ['bestRating', 'review', '@context', '@type', 'worstRating', 'author', 'aggregateRating', 'genre', 'url']
  for el in to_del:
    if el not in data2:
      continue
    else:
      del data2[el]

  category = soup.find('a', {'class': 'book__category d-sm-block d-none'})
  data2['category'] = category.string.strip()

  tags = soup.find_all('a', {'class': 'btn btn-outline-primary tag mt-2 mb-0'})
  tag_list = []
  for el in tags:
    tag_list.append(el.string.strip())
  tag_str = ', '.join(tag_list)
  data2['tags'] = tag_str

  wydawnictwo = soup.find('dd', {'class': 'd-block d-sm-none t-semi-bold'})
  wydawnictwo2 = wydawnictwo.find('a') 
  data2['publisher'] = wydawnictwo2.string
  data2['publisher']

  data2['url'] = str(url)

  def conv_pages_to_time():
    if 'numberOfPages' not in data2:
      return ""
    else:
      pages_min_time = int(data2['numberOfPages']) * 2
      pages_max_time = int(data2['numberOfPages']) * 3
      hours_min = pages_min_time // 60
      hours_max = pages_max_time // 60
      mins_min = pages_min_time - hours_min*60
      mins_max = pages_max_time - hours_max*60
      time = "%s h %s min - %s h %s min" % (hours_min, mins_min, hours_max, mins_max)
      return time

  #print("%s h %s min - %s h %s min" % (hours_min, mins_min, hours_max, mins_max))#return hours
  #time = "%s h %s min - %s h %s min" % (hours_min, mins_min, hours_max, mins_max)
  #conv_pages_to_time()
  #print(pages_min_time, pages_max_time, hours)
 
  data2['readingTime'] = conv_pages_to_time()

  dd_data = soup.find_all("dd")
  infobox = ['Seria:', 'Tytuł oryginału:', 'Data 1. wydania:', 'Data 1. wyd. pol.:']


  infobox2 = ['series', 'originalTitle', 'originPublish',  'firstPolishPublish']
  dd_data = soup.find_all("dd")
  dt_data = soup.find_all("dt")

  dt = []
  for el in dt_data:
    dt.append(el.text.strip())

  for item in dt_data:
    i = 0
    for el in infobox:
      if (el == item.string.strip()):
        data2[infobox2[infobox.index(item.string.strip())]] = dd_data[dt.index(item.string.strip())].text.strip()
      i +=1

  #print(data2)
  print("{} out of {} : {}".format(it, len(book_urls)+1, data2['book_title']))
  books.append(data2)
  df = df.append(data2,ignore_index=True)

print("Finished in: {}".format(datetime.now() - start_time))

2022-08-19 08:13:57.698992
753 out of 2 : Oczami radzieckiej zabawki
Finished in: 0:00:01.199240


In [ ]:
df

,image,book_title,name,datePublished,numberOfPages,ratingValue,ratingCount,category,publisher,series,readingTime,tags,originalTitle,firstPolishPublish,originPublish,isbn,inLanguage,url
0,https://s.lubimyczytac.pl/upload/books/141000/...,Oczami radzieckiej zabawki,Konstanty Usenko,2012-01-01,408,"8,0",103,reportaż,Czarne,Poza serią,13 h 36 min - 20 h 24 min,"punk, reportaż, nowa fala, ZSRR, electro, tech...",NaN,2012-01-01,NaN,9788375364668,polski,https://lubimyczytac.pl/ksiazka/141871/oczami-...


In [ ]:
df = df.sort_values(['name', 'book_title', 'datePublished'])
df = df.reset_index()
df_copy = df
df

,index,image,book_title,name,datePublished,numberOfPages,ratingValue,ratingCount,category,publisher,series,readingTime,tags,originalTitle,firstPolishPublish,originPublish,isbn,inLanguage,url
0,0,https://s.lubimyczytac.pl/upload/books/141000/...,Oczami radzieckiej zabawki,Konstanty Usenko,2012-01-01,408,"8,0",103,reportaż,Czarne,Poza serią,13 h 36 min - 20 h 24 min,"punk, reportaż, nowa fala, ZSRR, electro, tech...",NaN,2012-01-01,NaN,9788375364668,polski,https://lubimyczytac.pl/ksiazka/141871/oczami-...


In [ ]:
df_html = df
df_html = df_html.drop(['index'], axis=1)
df_html

,image,book_title,name,datePublished,numberOfPages,ratingValue,ratingCount,category,publisher,series,readingTime,tags,originalTitle,firstPolishPublish,originPublish,isbn,inLanguage,url
0,https://s.lubimyczytac.pl/upload/books/141000/...,Oczami radzieckiej zabawki,Konstanty Usenko,2012-01-01,408,"8,0",103,reportaż,Czarne,Poza serią,13 h 36 min - 20 h 24 min,"punk, reportaż, nowa fala, ZSRR, electro, tech...",NaN,2012-01-01,NaN,9788375364668,polski,https://lubimyczytac.pl/ksiazka/141871/oczami-...


In [ ]:
df_html.columns = ['Okładka', 'Tytuł', 'Autor', 'Data Publikacji', 'Liczba stron', 'Ocena', 'Liczba ocen', 'Kategoria', 'Wydawnictwo', 'Seria', 'Czas czytania', 'Tagi', 'Tytuł oryginału', 'Pierwsze wydanie pol.', 'Pierwsze wydanie oryg.', 'ISBN', 'Język', 'URL']
#df_html.columns = ['image', 'book_title','name', 'datePublished', 'numberOfPages', 'ratingValue', 'ratingCount', 'category', 
 #      'publisher', 'series',
  #     'readingTime', 'tags', 'originalTitle', 'firstPolishPublish', 'originPublish',
   #     'isbn', 'inLanguage', 'url']
df_html

,Okładka,Tytuł,Autor,Data Publikacji,Liczba stron,Ocena,Liczba ocen,Kategoria,Wydawnictwo,Seria,Czas czytania,Tagi,Tytuł oryginału,Pierwsze wydanie pol.,Pierwsze wydanie oryg.,ISBN,Język,URL
0,https://s.lubimyczytac.pl/upload/books/141000/...,Oczami radzieckiej zabawki,Konstanty Usenko,2012-01-01,408,"8,0",103,reportaż,Czarne,Poza serią,13 h 36 min - 20 h 24 min,"punk, reportaż, nowa fala, ZSRR, electro, tech...",NaN,2012-01-01,NaN,9788375364668,polski,https://lubimyczytac.pl/ksiazka/141871/oczami-...


In [ ]:
#df_html.shift()[2:]
df_html.index +=1
df_html

,Okładka,Tytuł,Autor,Data Publikacji,Liczba stron,Ocena,Liczba ocen,Kategoria,Wydawnictwo,Seria,Czas czytania,Tagi,Tytuł oryginału,Pierwsze wydanie pol.,Pierwsze wydanie oryg.,ISBN,Język,URL
1,https://s.lubimyczytac.pl/upload/books/141000/...,Oczami radzieckiej zabawki,Konstanty Usenko,2012-01-01,408,"8,0",103,reportaż,Czarne,Poza serią,13 h 36 min - 20 h 24 min,"punk, reportaż, nowa fala, ZSRR, electro, tech...",NaN,2012-01-01,NaN,9788375364668,polski,https://lubimyczytac.pl/ksiazka/141871/oczami-...


In [ ]:
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

def href_link(path):
    return '<a href="'+ path + '">' + path + '</a>'

df_html.to_html(escape=False, formatters=dict(Okładka=path_to_image_html,URL=href_link))
from IPython.core.display import HTML
HTML(df_html.to_html(escape=False,formatters=dict(Okładka=path_to_image_html,URL=href_link), classes='table table-stripped'))

,Okładka,Tytuł,Autor,Data Publikacji,Liczba stron,Ocena,Liczba ocen,Kategoria,Wydawnictwo,Seria,Czas czytania,Tagi,Tytuł oryginału,Pierwsze wydanie pol.,Pierwsze wydanie oryg.,ISBN,Język,URL
1,,Oczami radzieckiej zabawki,Konstanty Usenko,2012-01-01,408,"8,0",103,reportaż,Czarne,Poza serią,13 h 36 min - 20 h 24 min,"punk, reportaż, nowa fala, ZSRR, electro, techno, rap, kreskówki, wspomnienia",NaN,2012-01-01,NaN,9788375364668,polski,https://lubimyczytac.pl/ksiazka/141871/oczami-radzieckiej-zabawki


In [ ]:
print(df_html.to_html(escape=False,formatters=dict(Okładka=path_to_image_html,URL=href_link), classes='table table-stripped'))

<table border="1" class="dataframe table table-stripped">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Okładka</th>
      <th>Tytuł</th>
      <th>Autor</th>
      <th>Data Publikacji</th>
      <th>Liczba stron</th>
      <th>Ocena</th>
      <th>Liczba ocen</th>
      <th>Kategoria</th>
      <th>Wydawnictwo</th>
      <th>Seria</th>
      <th>Czas czytania</th>
      <th>Tagi</th>
      <th>Tytuł oryginału</th>
      <th>Pierwsze wydanie pol.</th>
      <th>Pierwsze wydanie oryg.</th>
      <th>ISBN</th>
      <th>Język</th>
      <th>URL</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1</th>
      <td><img src="https://s.lubimyczytac.pl/upload/books/141000/141871/352x500.jpg" width="60" ></td>
      <td>Oczami radzieckiej zabawki</td>
      <td>Konstanty Usenko</td>
      <td>2012-01-01</td>
      <td>408</td>
      <td>8,0</td>
      <td>103</td>
      <td>reportaż</td>
      <td>Czarne</td>
      <td>Poza serią</td>
      <td>13 h 36 min - 20 h 2

In [ ]:
df2 = df
df2.drop('index')
df2

KeyError: ignored

In [ ]:
list(df2.columns)

In [ ]:
df2 = df2.drop('index', axis=1)

In [ ]:
my_style = """background-color: rgba(0, 0, 0, 0);
border-bottom-color: rgb(0, 0, 0);
border-bottom-style: none;
border-bottom-width: 0px;
border-collapse: collapse;
border-image-outset: 0px;
border-image-repeat: stretch;
border-image-slice: 100%;
border-image-source: none;
border-image-width: 1;
border-left-color: rgb(0, 0, 0);
border-left-style: none;
border-left-width: 0px;
border-right-color: rgb(0, 0, 0);
border-right-style: none;
border-right-width: 0px;
border-top-color: rgb(0, 0, 0);
border-top-style: none;
border-top-width: 0px;
box-sizing: border-box;
color: rgb(0, 0, 0);
display: table;
font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
font-size: 12px;
height: 1675px;
line-height: 20px;
margin-left: 0px;
margin-right: 0px;
margin-top: 12px;
table-layout: fixed;
text-size-adjust: 100%;
width: 700px;
-webkit-border-horizontal-spacing: 0px;
-webkit-border-vertical-spacing: 0px;
-webkit-tap-highlight-color: rgba(0, 0, 0, 0);"""

In [ ]:
def HTML_with_style(df, style=None, random_id=None):
    from IPython.display import HTML
    import numpy as np
    import re

    df_html = df.to_html()

    if random_id is None:
        random_id = 'id%d' % np.random.choice(np.arange(1000000))

    if style is None:
        style = """
        <style>
            table#{random_id} {{color: blue}}
        </style>
        """.format(random_id=random_id)
    else:
        new_style = []
        s = re.sub(r'</?style>', '', style).strip()
        for line in s.split('\n'):
                line = line.strip()
                if not re.match(r'^table', line):
                    line = re.sub(r'^', 'table ', line)
                new_style.append(line)
        new_style = ['<style>'] + new_style + ['</style>']

        style = re.sub(r'table(#\S+)?', 'table#%s' % random_id, '\n'.join(new_style))

    df_html = re.sub(r'<table', r'<table id=%s ' % random_id, df_html)
    print(style, df_html)
    return HTML(style + df_html)


In [ ]:
HTML_with_style(df2, '<style>table {{{}}}</style>'.format(my_style))

# CLEAR DATAFRAME

In [ ]:
df = pd.DataFrame(None)

In [ ]:
#url = 'https://lubimyczytac.pl/ksiazka/4890328/turbopatriotyzm'
books = []
book = {}
df3 = pd.DataFrame()
start_time = datetime.now()
print(start_time)
for el in book_urls:
  it = el
  url = book_urls[el]
  result = requests.get(url)
  soup = bs4.BeautifulSoup(result.text, "html.parser")

  data = json.loads(soup.find('script', type='application/ld+json').text)

  data2 = {}
  data2.update(data)
  data2['book_title'] = data2['name']
  agr_rating = 'aggregateRating'
  if agr_rating not in data:
    continue
  else:
      data2.update(data['aggregateRating'])
  data2.update(data['author'])

  to_del = ['bestRating', 'review', '@context', '@type', 'worstRating', 'author', 'aggregateRating', 'genre', 'url']
  for el in to_del:
    if el not in data2:
      continue
  
  #start_time = datetime.now()
  #print( start_time)
  print("{} out of {} : {}".format(it, len(book_urls)+1, data2['book_title']))

print("Finished in: {}".format( datetime.now() - start_time))


2022-08-18 00:05:17.844899
1 out of 123 : Zachowuj się. Jak biologia wydobywa z nas to, co najgorsze, i to, co najlepsze
2 out of 123 : Zaraza
3 out of 123 : Gadka. W sześćdziesiąt języków dookoła Europy
4 out of 123 : Turbopatriotyzm
5 out of 123 : Wędrowny Zakład Fotograficzny
6 out of 123 : Kod kapitalizmu. Jak Gwiezdne wojny, Coca-Cola i Leo Messi kierują Twoim życiem
7 out of 123 : Powstanie umarłych. Historia pamięci 1944-2014
8 out of 123 : Babel. W dwadzieścia języków dookoła świata
9 out of 123 : Świat według Clarksona
10 out of 123 : Naprawdę?
11 out of 123 : Świat według Clarksona. Jeśli mógłbym dokończyć…
12 out of 123 : Świat według Clarksona 6. Tak jak mówiłem…
13 out of 123 : Co może pójść nie tak?
14 out of 123 : Przecież nie proszę o wiele...
15 out of 123 : W czym problem?
16 out of 123 : Doprowadzony do szału
17 out of 123 : Świat według Clarksona, część 3. Na litość boską!
18 out of 123 : Nie zatrzymasz mnie
19 out of 123 : I jeszcze jedno...
20 out of 123 : Księgi 

In [ ]:
fave_language = "Python"

print("I like coding in " + fave_language + " the most")

I like coding in Python the most


In [ ]:
len(book_urls)

122

In [ ]:
print(datetime.now() - start_time)

0:03:36.437165


In [ ]:
print(start_time)

2022-08-18 00:05:17.844899


In [ ]:
book_urls = {
1:'https://lubimyczytac.pl/ksiazka/4949256/zachowuj-sie-jak-biologia-wydobywa-z-nas-to-co-najgorsze-i-to-co-najlepsze',
2:'https://lubimyczytac.pl/ksiazka/297376/zaraza',
3:'https://lubimyczytac.pl/ksiazka/4973522/gadka-w-szescdziesiat-jezykow-dookola-europy',
4:'https://lubimyczytac.pl/ksiazka/4890328/turbopatriotyzm',
5:'https://lubimyczytac.pl/ksiazka/4889761/wedrowny-zaklad-fotograficzny',
6:'https://lubimyczytac.pl/ksiazka/4878403/kod-kapitalizmu-jak-gwiezdne-wojny-coca-cola-i-leo-messi-kieruja-twoim-zyciem',
7:'https://lubimyczytac.pl/ksiazka/314041/powstanie-umarlych-historia-pamieci-1944-2014',
8:'https://lubimyczytac.pl/ksiazka/4893518/babel-w-dwadziescia-jezykow-dookola-swiata',
9:'https://lubimyczytac.pl/ksiazka/37242/swiat-wedlug-clarksona',
10:'https://lubimyczytac.pl/ksiazka/4948581/naprawde',
11:'https://lubimyczytac.pl/ksiazka/4905516/swiat-wedlug-clarksona-jesli-moglbym-dokonczyc',
12:'https://lubimyczytac.pl/ksiazka/3988216/swiat-wedlug-clarksona-6-tak-jak-mowilem',
13:'https://lubimyczytac.pl/ksiazka/272749/co-moze-pojsc-nie-tak',
14:'https://lubimyczytac.pl/ksiazka/237438/przeciez-nie-prosze-o-wiele',
15:'https://lubimyczytac.pl/ksiazka/115172/w-czym-problem',
16:'https://lubimyczytac.pl/ksiazka/78234/doprowadzony-do-szalu',
17:'https://lubimyczytac.pl/ksiazka/57158/swiat-wedlug-clarksona-czesc-3-na-litosc-boska',
18:'https://lubimyczytac.pl/ksiazka/5729/nie-zatrzymasz-mnie',
19:'https://lubimyczytac.pl/ksiazka/49210/i-jeszcze-jedno',
20:'https://lubimyczytac.pl/ksiazka/163634/ksiegi-jakubowe',
21:'https://lubimyczytac.pl/ksiazka/4952609/szwajcaria-podroz-przez-raj-wymyslony',
22:'https://lubimyczytac.pl/ksiazka/3871955/mozesz-byc-bogaty-czyli-jak-szwajcarzy-osiagneli-sukces-gospodarczy',
23:'https://lubimyczytac.pl/ksiazka/4805016/rzeczozmeczenie',
24:'https://lubimyczytac.pl/ksiazka/4916710/tak-dzis-jemy-biografia-jedzenia',
25:'https://lubimyczytac.pl/ksiazka/4935904/betonoza-jak-sie-niszczy-polskie-miasta',
26:'https://lubimyczytac.pl/ksiazka/4886042/szybcy-jak-diabli-rywalizacja-forda-i-ferrari-o-zwyciestwo-w-wielkim-wyscigu-w-le-mans',
27:'https://lubimyczytac.pl/ksiazka/96979/wojny-reprodukcyjne-plec-wladza-a-przyszlosc-swiata',
28:'https://lubimyczytac.pl/ksiazka/196817/kuchnia-ormianska',
29:'https://lubimyczytac.pl/ksiazka/4847778/a-ja-zem-jej-powiedziala',
30:'https://lubimyczytac.pl/ksiazka/4902696/osobliwe-miejsca-ktore-odwiedzasz-kazdego-dnia',
31:'https://lubimyczytac.pl/ksiazka/4805363/koszmarne-istoty-ktore-spotykasz-kazdego-dnia',
32:'https://lubimyczytac.pl/ksiazka/4876470/w-co-wierza-polacy-sledztwo-w-sprawie-wrozek-jasnowidzow-szeptuch',
33:'https://lubimyczytac.pl/ksiazka/4985768/mroczna-polowa',
34:'https://lubimyczytac.pl/ksiazka/254758/dziewczyna-z-pociagu',
35:'https://lubimyczytac.pl/ksiazka/4784524/opowiesci-jerzego-urbana-kibica',
36:'https://lubimyczytac.pl/ksiazka/235716/marsjanin',
37:'https://lubimyczytac.pl/ksiazka/73930/rozkosze-podgladacza',
38:'https://lubimyczytac.pl/ksiazka/243241/klatwa-urbana',
39:'https://lubimyczytac.pl/ksiazka/242766/druga-klatwa-urbana',
40:'https://lubimyczytac.pl/ksiazka/93145/jajakobyly-spowiedz-zycia-jerzego-urbana',
41:'https://lubimyczytac.pl/ksiazka/115198/niestety-wszyscy-sie-znamy',
42:'https://lubimyczytac.pl/ksiazka/225527/prima-aprilis-towarzysze',
43:'https://lubimyczytac.pl/ksiazka/73748/alfabet-urbana',
44:'https://lubimyczytac.pl/ksiazka/4906797/wenus-w-futrze',
45:'https://lubimyczytac.pl/ksiazka/294813/sztukmistrz-z-lublina',
46:'https://lubimyczytac.pl/ksiazka/35409/jarmark-odmiencow',
47:'https://lubimyczytac.pl/ksiazka/74555/nowy-wspanialy-swiat',
48:'https://lubimyczytac.pl/ksiazka/4848863/nowy-wspanialy-swiat-30-lat-pozniej-raport-rozbieznosci',
49:'https://lubimyczytac.pl/ksiazka/141034/drzwi-percepcji-niebo-i-pieklo',
50:'https://lubimyczytac.pl/ksiazka/4862754/malowany-ptak',
51:'https://lubimyczytac.pl/ksiazka/4930333/gwiazda-polnocy-gwiazda-poludnia',
52:'https://lubimyczytac.pl/ksiazka/4930525/sto-dwadziescia-dni-sodomy-czyli-szkola-libertynizmu',
53:'https://lubimyczytac.pl/ksiazka/47662/my-dzieci-z-dworca-zoo',
54:'https://lubimyczytac.pl/ksiazka/4888689/pamietnik-ksiegarza',
55:'https://lubimyczytac.pl/ksiazka/246841/kurt-vonnegut-listy',
56:'https://lubimyczytac.pl/ksiazka/28544/zona-22',
57:'https://lubimyczytac.pl/ksiazka/312912/rzekomo-fajna-rzecz-ktorej-nigdy-wiecej-nie-zrobie',
58:'https://lubimyczytac.pl/ksiazka/4912720/trainspotting',
59:'https://lubimyczytac.pl/ksiazka/4278977/t2-trainspotting',
60:'https://lubimyczytac.pl/ksiazka/3823573/trainspotting-zero',
61:'https://lubimyczytac.pl/ksiazka/4915988/zero-zero-zero-jak-kokaina-rzadzi-swiatem',
62:'https://lubimyczytac.pl/ksiazka/4997332/do-you-read-me-bookstores-around-the-world',
63:'https://lubimyczytac.pl/ksiazka/4973315/paskudnik-warszawski',
64:'https://lubimyczytac.pl/ksiazka/154347/lwow-1918-1919',
65:'https://lubimyczytac.pl/ksiazka/62361/adolf-hitler-biografia-fuhrera',
66:'https://lubimyczytac.pl/ksiazka/4769955/ta-nasza-mlodosc-jak-dorastalo-sie-w-prl',
67:'https://lubimyczytac.pl/ksiazka/7176/ryszard-kuklinski-zycie-scisle-tajne',
68:'https://lubimyczytac.pl/ksiazka/297294/czlowiek-to-czlowiek-a-smierc-to-smierc',
69:'https://lubimyczytac.pl/ksiazka/293762/dzieci-z-bullerbyn',
70:'https://lubimyczytac.pl/ksiazka/107521/socjologia-rodziny-ewolucja-historia-zroznicowanie',
71:'https://lubimyczytac.pl/ksiazka/4943173/piknik-na-skraju-drogi-i-inne-utwory',
72:'https://lubimyczytac.pl/ksiazka/4914319/tragedia-na-przeleczy-diatlowa-historia-bez-konca',
73:'https://lubimyczytac.pl/ksiazka/4904847/przyszlo-nam-tu-zyc-reportaze-z-rosji',
74:'https://lubimyczytac.pl/ksiazka/4904047/milion-malych-kawalkow',
75:'https://lubimyczytac.pl/ksiazka/267343/kompleks-palacu-zycie-spoleczne-stalinowskiego-wiezowca-w-kapitalistycznej-warszawie',
76:'https://lubimyczytac.pl/ksiazka/4930530/chemia-smierci',
77:'https://lubimyczytac.pl/ksiazka/4860810/factfulness-dlaczego-swiat-jest-lepszy-niz-myslimy-czyli-jak-stereotypy-zastapic-realna-wiedza',
78:'https://lubimyczytac.pl/ksiazka/4886342/wladcy-jedzenia-jak-przemysl-spozywczy-niszczy-planete',
79:'https://lubimyczytac.pl/ksiazka/4860477/jak-sfalszowac-wybory',
80:'https://lubimyczytac.pl/ksiazka/189696/jezyk-rzeczy-w-jaki-sposob-przedmioty-nas-uwodza',
81:'https://lubimyczytac.pl/ksiazka/4933084/zdrowe-zeby',
82:'https://lubimyczytac.pl/ksiazka/4724047/451-fahrenheita',
83:'https://lubimyczytac.pl/ksiazka/5001198/mistrz-i-malgorzata',
84:'https://lubimyczytac.pl/ksiazka/3970926/niedziela-ktora-zdarzyla-sie-w-srode',
85:'https://lubimyczytac.pl/ksiazka/4906483/ostatni-brzeg',
86:'https://lubimyczytac.pl/ksiazka/4817475/perfekcyjna-maszyna-do-czytania-o-ergonomii-ksiazki',
87:'https://lubimyczytac.pl/ksiazka/4925851/homo-politicus-sapiens-biologiczne-aspekty-politycznej-gry',
88:'https://lubimyczytac.pl/ksiazka/179360/system-prawny-szwajcarii-historia-i-wspolczesnosc',
89:'https://lubimyczytac.pl/ksiazka/218164/krotka-historia-ksiazek-zakazanych',
90:'https://lubimyczytac.pl/ksiazka/169044/parlament-szwajcarii',
91:'https://lubimyczytac.pl/ksiazka/4914195/pokaz-mi-swoja-biblioteke',
92:'https://lubimyczytac.pl/ksiazka/4857338/chleb-techniki-wypieku-przepisy-wskazowki-wydanie-nowe-uzupelnione',
93:'https://lubimyczytac.pl/ksiazka/309961/swiss-made-nieznana-historia-szwajcarskiego-sukcesu',
94:'https://lubimyczytac.pl/ksiazka/4840610/szwajcaria-i-nazisci-jak-alpejska-republika-przetrwala-w-cieniu-iii-rzeszy',
95:'https://lubimyczytac.pl/ksiazka/162254/historia-szwajcarii',
96:'https://lubimyczytac.pl/ksiazka/4871397/fenomen-szwajcarii-wobec-unii-europejskiej',
97:'https://lubimyczytac.pl/ksiazka/90347/cel---szwajcaria',
98:'https://lubimyczytac.pl/ksiazka/4937303/instagram-bez-filtra',
99:'https://lubimyczytac.pl/ksiazka/4899851/wszyscy-klamia-big-data-nowe-dane-i-wszystko-co-internet-moze-nam-powiedziec-o-tym-kim-naprawde',
100:'https://lubimyczytac.pl/ksiazka/4238739/ku-urbanologii',
101:'https://lubimyczytac.pl/ksiazka/4849913/the-world-atlas-of-coffee---from-beans-to-brewing---coffees-explored-explained-and-enjoyed',
102:'https://lubimyczytac.pl/ksiazka/4943371/ja-i-moje-przyjaciolki-idiotki',
103:'https://lubimyczytac.pl/ksiazka/4853646/polska-moze-byc-lepsza-kulisy-polskiej-dyplomacji',
104:'https://lubimyczytac.pl/ksiazka/4929132/terapia-z-uzyciem-mdma-i-innych-entaktogenow',
105:'https://lubimyczytac.pl/ksiazka/5027576/miedzianka-historia-znikania',
106:'https://lubimyczytac.pl/ksiazka/4888370/nie-zdaze',
107:'https://lubimyczytac.pl/ksiazka/4995859/cztery-wymiary-architektury',
108:'https://lubimyczytac.pl/ksiazka/4098636/ksiazka-najpotezniejszy-przedmiot-naszych-czasow-zbadany-od-deski-do-deski',
109:'https://lubimyczytac.pl/ksiazka/4914929/osobisty-przewodnik-po-pradze',
110:'https://lubimyczytac.pl/ksiazka/4847951/dwa-tysiace-instrukcja-obslugi-polskiej-urbanizacji-w-xxi-wieku',
111:'https://lubimyczytac.pl/ksiazka/4946698/codziennosc-nudy-dom-jako-przestrzen-produktywna-i-nieproduktywna',
112:'https://lubimyczytac.pl/ksiazka/4308457/jezyk-miast',
113:'https://lubimyczytac.pl/ksiazka/4871916/dobrze-nastrojone-miasto',
114:'https://lubimyczytac.pl/ksiazka/4913667/najlepsze-miasto-swiata-warszawa-w-odbudowie-1944-1949',
115:'https://lubimyczytac.pl/ksiazka/4805634/dom-polski-mebloscianka-z-pikasami',
116:'https://lubimyczytac.pl/ksiazka/4928498/13-pieter',
117:'https://lubimyczytac.pl/ksiazka/160661/arps-architektura-arseniusza-romanowicza-i-piotra-szymaniaka',
118:'https://lubimyczytac.pl/ksiazka/4999291/atlas-ladow-niebylych-najwieksze-mity-zmyslenia-i-pomylki-kartografow',
119:'https://lubimyczytac.pl/ksiazka/256836/palac-biografia-intymna',
120:'https://lubimyczytac.pl/ksiazka/4843891/chaos-warszawa-porzadki-przestrzenne-polskiego-kapitalizmu',
121:'https://lubimyczytac.pl/ksiazka/4628118/sto-lat-samotnosci',
122:'https://lubimyczytac.pl/ksiazka/4807144/milosc-w-czasach-zarazy',
123:'https://lubimyczytac.pl/ksiazka/154212/pulapki-myslenia-o-mysleniu-szybkim-i-wolnym',
124:'https://lubimyczytac.pl/ksiazka/4800628/1000-slow',
125:'https://lubimyczytac.pl/ksiazka/256283/500-zdan-polskich',
126:'https://lubimyczytac.pl/ksiazka/4892768/zapomniane-slowa-z-prl-u',
127:'https://lubimyczytac.pl/ksiazka/220979/wszystko-zalezy-od-przyimka',
128:'https://lubimyczytac.pl/ksiazka/4900230/zwierzyniec',
129:'https://lubimyczytac.pl/ksiazka/4892846/diaboliada-fatalne-jaja---adaptacja-klasyki-z-cwiczeniami---jezyk-rosyjski',
130:'https://lubimyczytac.pl/ksiazka/4012728/sztuka-kochania',
131:'https://lubimyczytac.pl/ksiazka/4818570/sztuka-obslugi-penisa',
132:'https://lubimyczytac.pl/ksiazka/4878771/prosze-sie-nie-bac-zaraz-pan-zasnie',
133:'https://lubimyczytac.pl/ksiazka/4857319/ginekolodzy-tajemnice-gabinetow',
134:'https://lubimyczytac.pl/ksiazka/4846628/scigajac-krzyk',
135:'https://lubimyczytac.pl/ksiazka/4937898/gdy-rosliny-snia',
136:'https://lubimyczytac.pl/ksiazka/236453/w-pogoni-w-ucieczce-narkotyki-i-narkomania',
137:'https://lubimyczytac.pl/ksiazka/4877377/narodziny-pigulki',
138:'https://lubimyczytac.pl/ksiazka/4874676/nowe-zycie-jak-najwieksze-umysly-wszechczasow-odkryly-skad-sie-biora-dzieci',
139:'https://lubimyczytac.pl/ksiazka/75703/historia-nagosci',
140:'https://lubimyczytac.pl/ksiazka/4976353/wstep-do-psychoanalizy',
141:'https://lubimyczytac.pl/ksiazka/127812/problemy-wspolczesnego-erotyzmu',
142:'https://lubimyczytac.pl/ksiazka/146654/rozwoj-erotyczny',
143:'https://lubimyczytac.pl/ksiazka/5022378/metoda-billingsa-sterowanie-plodnoscia-bez-pomocy-srodkow-farmaceutycznych-i-mechanicznych',
144:'https://lubimyczytac.pl/ksiazka/4859052/sexedpl-rozmowy-anji-rubik-o-dojrzewaniu-milosci-i-seksie',
145:'https://lubimyczytac.pl/ksiazka/5009337/on-pytania-intymne-ona-pytania-intymne',
146:'https://lubimyczytac.pl/ksiazka/288212/ginekolodzy',
147:'https://lubimyczytac.pl/ksiazka/243135/wenus-bez-futra',
148:'https://lubimyczytac.pl/ksiazka/4861270/homo-nie-calkiem-sapiens',
149:'https://lubimyczytac.pl/ksiazka/4874551/ikony-popkultury-a-prawo-wlasnosci-intelektualnej-jak-znani-i-slawni-chronia-swoje-prawa',
150:'https://lubimyczytac.pl/ksiazka/4559586/wysoko-wrazliwi-jak-funkcjonowac-w-swiecie-ktory-nas-przytlacza',
151:'https://lubimyczytac.pl/ksiazka/105062/eliksir-hofmanna-lsd-i-nowe-eleuzis',
152:'https://lubimyczytac.pl/ksiazka/4893058/osaczeni-przez-liczby-o-algorytmach-ktore-kontroluja-nasze-zycie-od-facebooka-i-googla-po-fake-ne',
153:'https://lubimyczytac.pl/ksiazka/4969515/sex-disco-i-kasety-video-polska-lat-90',
154:'https://lubimyczytac.pl/ksiazka/257943/operacja-paperclip',
155:'https://lubimyczytac.pl/ksiazka/4855264/osobowosc-na-talerzu-kim-sa-zwierzeta-ktore-zjadamy',
156:'https://lubimyczytac.pl/ksiazka/4850948/dziewczyny-z-wolynia',
157:'https://lubimyczytac.pl/ksiazka/4808087/kobiety-ze-lwowa',
158:'https://lubimyczytac.pl/ksiazka/143939/pod-berlem-habsburgow-zagadnienia-narodowosciowe',
159:'https://lubimyczytac.pl/ksiazka/4899249/smog-diesle-kopciuchy-kominy-czyli-dlaczego-w-polsce-nie-da-sie-oddychac',
160:'https://lubimyczytac.pl/ksiazka/4847012/hanba-opowiesci-o-polskiej-zdradzie',
161:'https://lubimyczytac.pl/ksiazka/232695/rewolucja-paranoiczno-krytyczna',
162:'https://lubimyczytac.pl/ksiazka/158986/gordon-ramsay-na-samym-szczycie',
163:'https://lubimyczytac.pl/ksiazka/128328/serge-gainsbourg',
164:'https://lubimyczytac.pl/ksiazka/4685745/przewodnik-wedrowca-sztuka-odczytywania-znakow-natury',
165:'https://lubimyczytac.pl/ksiazka/4821061/jak-czytac-wode-wskazowki-i-znaki-ukryte-w-morzach-jeziorach-rzekach-i-kaluzach',
166:'https://lubimyczytac.pl/ksiazka/195835/barszcz-ukrainski',
167:'https://lubimyczytac.pl/ksiazka/4960178/trzecia-rzesza-na-haju-narkotyki-w-hitlerowskich-niemczech',
168:'https://lubimyczytac.pl/ksiazka/98785/historia-boga-4000-lat-dziejow-boga-w-judaizmie-chrzescijanstwie-i-islamie',
169:'https://lubimyczytac.pl/ksiazka/4901596/grzechy-quot-paryza-polnocy-quot-mroczne-zycie-przedwojennej-warszawy',
170:'https://lubimyczytac.pl/ksiazka/234383/atomowy-szpieg-ryszard-kuklinski-i-wojna-wywiadow',
171:'https://lubimyczytac.pl/ksiazka/4958881/praga-i-jej-bazary-opowiadania-i-anegdoty-o-warszawskiej-pradze-z-lat-1950-1970',
172:'https://lubimyczytac.pl/ksiazka/233410/genialni-lwowska-szkola-matematyczna',
173:'https://lubimyczytac.pl/ksiazka/297463/pogranicze-podroz-przez-historie-ukrainy-988-2015',
174:'https://lubimyczytac.pl/ksiazka/4953324/calkiem-zwyczajny-kraj-historia-polski-bez-martyrologii',
175:'https://lubimyczytac.pl/ksiazka/4896524/ostatnie-lata-polskiego-lwowa',
176:'https://lubimyczytac.pl/ksiazka/295944/kto-wydal-wyrok-na-miasto-plany-operacyjne-zwz-ak-1940-1944-i-sposoby-ich-realizacji',
177:'https://lubimyczytac.pl/ksiazka/5736/powojnie-historia-europy-od-roku-1945',
178:'https://lubimyczytac.pl/ksiazka/4883499/zapomniany-wiek-xx-retrospekcje',
179:'https://lubimyczytac.pl/ksiazka/4892602/tango-milonga-czyli-co-nam-zostalo-z-tamtych-lat',
180:'https://lubimyczytac.pl/ksiazka/88169/tamten-lwow-t-1-oblicze-miasta',
181:'https://lubimyczytac.pl/ksiazka/202885/tamten-lwow-tom-2-ulice-i-place',
182:'https://lubimyczytac.pl/ksiazka/206955/tamten-lwow-tom-3-swiatynie-gmachy-pomniki',
183:'https://lubimyczytac.pl/ksiazka/235389/tamten-lwow-tom-4-my-lwowianie',
184:'https://lubimyczytac.pl/ksiazka/284954/tamten-lwow-tom-5-zycie-miasta',
185:'https://lubimyczytac.pl/ksiazka/5008853/polska-rosja-historia-obsesji-obsesja-historii',
186:'https://lubimyczytac.pl/ksiazka/4467586/the-gates-of-europe-a-history-of-ukraine',
187:'https://lubimyczytac.pl/ksiazka/4938304/historia-swiata-na-czterech-lapach',
188:'https://lubimyczytac.pl/ksiazka/270028/ocalalem-z-buchenwaldu-moja-droga-przez-pieklo',
189:'https://lubimyczytac.pl/ksiazka/4924576/obled-44-czyli-jak-polacy-zrobili-prezent-stalinowi-wywolujac-powstanie-warszawskie',
190:'https://lubimyczytac.pl/ksiazka/4887795/wolyn-zdradzony-czyli-jak-dowodztwo-ak-porzucilo-polakow-na-pastwe-upa',
191:'https://lubimyczytac.pl/ksiazka/4819827/skazy-na-pancerzach-czarne-karty-epopei-zolnierzy-wykletych',
192:'https://lubimyczytac.pl/ksiazka/304763/zydzi-opowiesci-niepoprawne-politycznie',
193:'https://lubimyczytac.pl/ksiazka/4865638/zydzi-2-opowiesci-niepoprawne-politycznie',
194:'https://lubimyczytac.pl/ksiazka/4955896/alianci-opowiesci-niepoprawne-politycznie',
195:'https://lubimyczytac.pl/ksiazka/4742226/niemcy-opowiesci-niepoprawne-politycznie-cz-iii',
196:'https://lubimyczytac.pl/ksiazka/85173/europa-rozprawa-historyka-z-historia',
197:'https://lubimyczytac.pl/ksiazka/4890977/obcym-alfabetem-jak-ludzie-kremla-i-pis-zagrali-podsluchami',
198:'https://lubimyczytac.pl/ksiazka/6574/historia-ukrainy',
199:'https://lubimyczytac.pl/ksiazka/4892065/lwow-kres-iluzji-opowiesc-o-pogromie-listopadowym-1918',
200:'https://lubimyczytac.pl/ksiazka/52324/opowiadania-odeskie',
201:'https://lubimyczytac.pl/ksiazka/102475/lwow-cuda-polski',
202:'https://lubimyczytac.pl/ksiazka/4952359/lukaszenka-niedoszly-car-rosji',
203:'https://lubimyczytac.pl/ksiazka/4924615/czas-wolny-w-prl',
204:'https://lubimyczytac.pl/ksiazka/4912039/wielki-nastepca-niebianskie-przeznaczenie-blyskotliwego-towarzysza-kim-dzong-una',
205:'https://lubimyczytac.pl/ksiazka/4935193/trolle-putina-prawdziwe-historie-z-frontow-rosyjskiej-wojny-informacyjnej',
206:'https://lubimyczytac.pl/ksiazka/4917670/krotka-historia-upa-dla-polakow-czy-historycy-nas-pogodza',
207:'https://lubimyczytac.pl/ksiazka/4955400/idea-galicji-historia-i-fantazja-w-kulturze-politycznej-habsburgow',
#208:'https://lubimyczytac.pl/ksiazka/244064/historia-bomby-atomowej-stany-zjednoczone-rzesza-niemiecka-zwiazek-radziecki-fakty-relacje',
209:'https://lubimyczytac.pl/ksiazka/4431702/azyl-zapiski-stalkera',
210:'https://lubimyczytac.pl/ksiazka/4856670/cena-niewazkosci-kulisy-lotu-polaka-w-kosmos',
211:'https://lubimyczytac.pl/ksiazka/4919159/gawedy-o-wilkach-i-innych-zwierzetach',
212:'https://lubimyczytac.pl/ksiazka/4821140/wilczyca-opowiesc-o-sile-natury-i-ludzkiej-obsesji-zabijania',
213:'https://lubimyczytac.pl/ksiazka/4878297/madrosc-wilkow',
214:'https://lubimyczytac.pl/ksiazka/3715112/przedziwny-czlowiek-sekretne-zycie-paula-diraca-geniusza-mechaniki-kwantowej',
215:'https://lubimyczytac.pl/ksiazka/4875139/enrico-fermi-ostatni-czlowiek-ktory-wiedzial-wszystko-zycie-i-czasy-ojca-ery-atomowej',
216:'https://lubimyczytac.pl/ksiazka/4874648/blysk-potega-przeczucia',
217:'https://lubimyczytac.pl/ksiazka/4909518/dawid-i-goliat-jak-skazani-na-niepowodzenie-moga-pokonac-gigantow',
218:'https://lubimyczytac.pl/ksiazka/4874650/poza-schematem-sekrety-ludzi-sukcesu',
219:'https://lubimyczytac.pl/ksiazka/4874651/punkt-przelomowy-o-malych-przyczynach-wielkich-zmian',
220:'https://lubimyczytac.pl/ksiazka/4909052/jak-rozmawiac-z-nieznajomymi-co-powinnismy-wiedziec-o-ludziach-o-ktorych-nic-nie-wiemy',
221:'https://lubimyczytac.pl/ksiazka/4818969/slodziutki-biografia-cukru',
222:'https://lubimyczytac.pl/ksiazka/4929628/farmagedon-rzeczywisty-koszt-taniego-miesa',
223:'https://lubimyczytac.pl/ksiazka/181080/zjadanie-zwierzat',
224:'https://lubimyczytac.pl/ksiazka/295333/w-rzeczy-samej-osobliwe-historie-wspanialych-materialow-ktore-nadaja-ksztalt-naszemu-swiatu',
225:'https://lubimyczytac.pl/ksiazka/88203/lekarze-naukowcy-szarlatani-od-przerazonego-pacjenta-do-swiadomego-konsumenta',
226:'https://lubimyczytac.pl/ksiazka/4860248/zdrowas-mario-reportaze-o-medycznej-marihuanie',
227:'https://lubimyczytac.pl/ksiazka/4874610/the-radium-girls-mroczna-historia-promiennych-kobiet-ameryki',
228:'https://lubimyczytac.pl/ksiazka/4898466/jak-wynalezc-wszystko-cala-wiedza-ludzkosci-w-jednej-ksiazce',
229:'https://lubimyczytac.pl/ksiazka/249064/wahadlo-foucaulta',
230:'https://lubimyczytac.pl/ksiazka/4988783/zycie-instrukcja-obslugi',
231:'https://lubimyczytac.pl/ksiazka/55095/czarnobyl-spowiedz-reportera',
232:'https://lubimyczytac.pl/ksiazka/182049/dziewczyny-atomowe',
233:'https://lubimyczytac.pl/ksiazka/4822429/katastrofa-w-czarnobylu-przyczyny-wybuchu-reaktora---obalanie-mitow',
234:'https://lubimyczytac.pl/ksiazka/226432/bron-jadrowa',
235:'https://lubimyczytac.pl/ksiazka/312354/sekretne-zycie-drzew',
236:'https://lubimyczytac.pl/ksiazka/4254758/duchowe-zycie-zwierzat',
237:'https://lubimyczytac.pl/ksiazka/4807074/nieznane-wiezi-natury',
238:'https://lubimyczytac.pl/ksiazka/4902631/na-ratunek-rozmowy-o-zwierzetach-naturze-i-przyszlosci-naszej-planety',
239:'https://lubimyczytac.pl/ksiazka/309689/pies-na-diecie-barf-zdrowe-i-naturalne-jedzenie-dla-twojego-pupila',
240:'https://lubimyczytac.pl/ksiazka/4946227/pies-na-diecie-barf-komponowanie-i-modyfikowanie-diety-barf-na-podstawie-stanu-zdrowia-i-wynikow',
241:'https://lubimyczytac.pl/ksiazka/4931828/dramat-zwierzat-domowych-weterynarz-patolog-o-cichych-cierpieniach-naszych-domowych-pupili',
242:'https://lubimyczytac.pl/ksiazka/4939975/rozmerdane-naukowe-podstawy-psiego-szczescia',
243:'https://lubimyczytac.pl/ksiazka/4413025/david-lynch',
244:'https://lubimyczytac.pl/ksiazka/4800252/w-pogoni-za-wielka-ryba-medytacja-swiadomosc-kreatywnosc',
245:'https://lubimyczytac.pl/ksiazka/4857404/czyste-mieso-jak-hodowla-miesa-bez-zwierzat-zrewolucjonizuje-twoj-obiad-i-caly-swiat',
246:'https://lubimyczytac.pl/ksiazka/4888560/manson-cia-narkotyki-mroczne-tajemnice-hollywood',
247:'https://lubimyczytac.pl/ksiazka/4937395/eksperyment-utopia',
248:'https://lubimyczytac.pl/ksiazka/4848978/dzisiaj-narysujemy-smierc',
249:'https://lubimyczytac.pl/ksiazka/250652/zle-ma-sie-kraj-rozprawa-o-naszych-wspolczesnych-bolaczkach',
250:'https://lubimyczytac.pl/ksiazka/4857894/obywatel-coke-kapitalizm-wedlug-coca-coli',
251:'https://lubimyczytac.pl/ksiazka/242216/czerwony-alert',
252:'https://lubimyczytac.pl/ksiazka/29507/absurdy-prl-u-antologia',
253:'https://lubimyczytac.pl/ksiazka/17257/absurdy-prl-u-2-antologia',
254:'https://lubimyczytac.pl/ksiazka/4810218/prawdziwa-historia-mcdonald-s-wspomnienia-zalozyciela',
255:'https://lubimyczytac.pl/ksiazka/4880785/zbuntowany-nowy-jork-wolnosc-w-czasach-prohibicji',
256:'https://lubimyczytac.pl/ksiazka/175764/moja-trojka',
257:'https://lubimyczytac.pl/ksiazka/4884035/zdarzylo-sie-naprawde-opowiesci-reporterskie',
258:'https://lubimyczytac.pl/ksiazka/179612/matrioszka-rosja-i-jastrzab',
259:'https://lubimyczytac.pl/ksiazka/224228/literacki-melanz-z-pragi',
260:'https://lubimyczytac.pl/ksiazka/4896078/pamiec-nieulotna',
261:'https://lubimyczytac.pl/ksiazka/4892615/koderzy-opowiesc-o-ludziach-ktorzy-zmienili-nasz-swiat',
262:'https://lubimyczytac.pl/ksiazka/4883373/rakietowi-miliarderzy-elon-musk-jeff-bezos-i-nowy-wyscig-kosmiczny',
263:'https://lubimyczytac.pl/ksiazka/301421/zabojcy-w-imie-republiki',
264:'https://lubimyczytac.pl/ksiazka/301609/panstwu-pierwszym-opowiem-o-trojce',
265:'https://lubimyczytac.pl/ksiazka/73334/rockmann-czyli-jak-nie-zostalem-saksofonista',
266:'https://lubimyczytac.pl/ksiazka/20290/dowcipy-prl-u-antologia',
267:'https://lubimyczytac.pl/ksiazka/4898258/pastrami',
268:'https://lubimyczytac.pl/ksiazka/4882847/emigracja',
269:'https://lubimyczytac.pl/ksiazka/145664/spowiedz-heretyka-sacrum-profanum',
270:'https://lubimyczytac.pl/ksiazka/4629944/polska-odwraca-oczy',
271:'https://lubimyczytac.pl/ksiazka/4893301/polski-bajer-disco-polo-i-lata-90',
272:'https://lubimyczytac.pl/ksiazka/4923422/korea-polnocna-dziennik-podrozy',
273:'https://lubimyczytac.pl/ksiazka/166913/gogol-w-czasach-google-a',
274:'https://lubimyczytac.pl/ksiazka/4900753/o-kobiecie-pracujacej-dlaczego-mniej-zarabia-chociaz-wiecej-pracuje',
275:'https://lubimyczytac.pl/ksiazka/207519/naga-prawda-o-seksmisji',
276:'https://lubimyczytac.pl/ksiazka/4056922/kultura-w-plot',
277:'https://lubimyczytac.pl/ksiazka/246319/cesarz-disco-polo',
278:'https://lubimyczytac.pl/ksiazka/154137/the-rolling-stones-za-zelazna-kurtyna-warszawa-1967',
279:'https://lubimyczytac.pl/ksiazka/4909447/selfie-autoportret',
280:'https://lubimyczytac.pl/ksiazka/239099/daft-punk-podroz-do-wnetrza-piramidy',
281:'https://lubimyczytac.pl/ksiazka/4878403/kod-kapitalizmu-jak-gwiezdne-wojny-coca-cola-i-leo-messi-kieruja-twoim-zyciem',
282:'https://lubimyczytac.pl/ksiazka/4864816/mitologia-wspolczesna',
283:'https://lubimyczytac.pl/ksiazka/294700/jerry-ego-lee-lewisa-opowiesc-o-wlasnym-zyciu',
284:'https://lubimyczytac.pl/ksiazka/4818020/please-kill-me-punkowa-historia-punka',
285:'https://lubimyczytac.pl/ksiazka/308207/krew-i-burza-historia-z-dzikiego-zachodu',
286:'https://lubimyczytac.pl/ksiazka/4821335/wyniosle-wieze-al-kaida-i-atak-na-ameryke',
287:'https://lubimyczytac.pl/ksiazka/4964704/wszystko-za-zycie',
288:'https://lubimyczytac.pl/ksiazka/4658774/niebo-jest-nasze-milosc-i-terror-w-zlotym-wieku-piractwa-powietrznego',
289:'https://lubimyczytac.pl/ksiazka/178740/nowy-jork-od-mannahatty-do-ground-zero',
290:'https://lubimyczytac.pl/ksiazka/4571860/bracia-wright',
291:'https://lubimyczytac.pl/ksiazka/4883342/ksiezycowy-pyl-w-poszukiwaniu-ludzi-ktorzy-spadli-na-ziemie',
292:'https://lubimyczytac.pl/ksiazka/5023658/dreamland-opiatowa-epidemia-w-usa',
293:'https://lubimyczytac.pl/ksiazka/4969478/hawana-podzwrotnikowe-delirium',
294:'https://lubimyczytac.pl/ksiazka/3676075/odessa-geniusz-i-smierc-w-miescie-snow',
295:'https://lubimyczytac.pl/ksiazka/208705/leksykon-miast-intymnych-swobodny-podrecznik-do-geopoetyki-i-kosmopolityki',
296:'https://lubimyczytac.pl/ksiazka/4814587/granice-marzen-o-panstwach-nieuznawanych',
297:'https://lubimyczytac.pl/ksiazka/3675766/dzieci-norwegii-o-panstwie-nad-opiekunczym',
298:'https://lubimyczytac.pl/ksiazka/291777/armenia-karawany-smierci',
299:'https://lubimyczytac.pl/ksiazka/264849/mur-12-kawalkow-o-berlinie',
300:'https://lubimyczytac.pl/ksiazka/4846651/urobieni-reportaze-o-pracy',
301:'https://lubimyczytac.pl/ksiazka/4861311/niepowtarzalny-urok-likwidacji-reportaze-z-polski-lat-90',
302:'https://lubimyczytac.pl/ksiazka/256528/jadro-dziwnosci-nowa-rosja',
303:'https://lubimyczytac.pl/ksiazka/247406/wilki-zyja-poza-prawem-jak-janukowycz-przegral-ukraine',
304:'https://lubimyczytac.pl/ksiazka/3675875/krysztalowy-fortepian-zdrady-i-zwyciestwa-petra-poroszenki',
305:'https://lubimyczytac.pl/ksiazka/4883200/na-marne',
306:'https://lubimyczytac.pl/ksiazka/298112/kazdy-zostal-czlowiekiem',
307:'https://lubimyczytac.pl/ksiazka/225348/biale-zimna-wyspa-spitsbergen',
308:'https://lubimyczytac.pl/ksiazka/294707/zabic-smoka-ukrainskie-rewolucje',
309:'https://lubimyczytac.pl/ksiazka/55540/diabel-tkwi-w-serze',
310:'https://lubimyczytac.pl/ksiazka/213965/w-gazetach-tego-nie-napisza',
311:'https://lubimyczytac.pl/ksiazka/225642/zwrotnik-ukraina',
312:'https://lubimyczytac.pl/ksiazka/4855657/naddniestrze-terror-tozsamosci',
313:'https://lubimyczytac.pl/ksiazka/82312/odessa-transfer-reportaze-znad-morza-czarnego',
314:'https://lubimyczytac.pl/ksiazka/260343/znikajaca-europa',
315:'https://lubimyczytac.pl/ksiazka/4985361/petersburg-miasto-snu',
316:'https://lubimyczytac.pl/ksiazka/4777432/po-galicji-o-chasydach-huculach-polakach-i-rusinach-imaginacyjna-podroz-po-galicji-wschodniej-i',
317:'https://lubimyczytac.pl/ksiazka/4896059/donikad-podroze-na-skraj-rosji',
318:'https://lubimyczytac.pl/ksiazka/104460/dwanascioro-niepokornych-portrety-nowych-rosyjskich-dysydentow',
319:'https://lubimyczytac.pl/ksiazka/294699/hen-na-polnocy-norwegii',
320:'https://lubimyczytac.pl/ksiazka/227739/czasy-secondhand-koniec-czerwonego-czlowieka',
321:'https://lubimyczytac.pl/ksiazka/4749503/ksiezyc-z-peweksu-o-luksusie-w-prl',
322:'https://lubimyczytac.pl/ksiazka/159647/naga-wyspa',
323:'https://lubimyczytac.pl/ksiazka/4880281/kim-dzong-un-szkic-portretu-dyktatora',
324:'https://lubimyczytac.pl/ksiazka/123413/z-nowego-wspanialego-swiata',
325:'https://lubimyczytac.pl/ksiazka/109247/swiatu-nie-mamy-czego-zazdroscic-zwyczajne-losy-mieszkancow-korei-polnocnej',
326:'https://lubimyczytac.pl/ksiazka/4800034/efekt-lucyfera-dlaczego-dobrzy-ludzie-czynia-zlo',
327:'https://lubimyczytac.pl/ksiazka/246208/psychologia-i-zycie',
328:'https://lubimyczytac.pl/ksiazka/285346/wywieranie-wplywu-na-ludzi-teoria-i-praktyka',
329:'https://lubimyczytac.pl/ksiazka/4032155/pre-swazja-jak-w-pelni-wykorzystac-techniki-wplywu-spolecznego',
330:'https://lubimyczytac.pl/ksiazka/92926/medaliony',
331:'https://lubimyczytac.pl/ksiazka/4853071/rzeznicy-i-lekarze-makabryczny-swiat-medycyny-i-rewolucja-josepha-listera',
332:'https://lubimyczytac.pl/ksiazka/4818557/zbrodnie-prawie-doskonale-policyjne-archiwum-x',
333:'https://lubimyczytac.pl/ksiazka/4968868/zrozumiec-szalenstwo-najglosniejszy-eksperyment-w-historii-psychiatrii-i-mroczna-prawda-ktora-sie',
334:'https://lubimyczytac.pl/ksiazka/166649/dowod-prawdziwa-historia-neurochirurga-ktory-przekroczyl-granice-smierci-i-odkryl-niebo',
335:'https://lubimyczytac.pl/ksiazka/4938891/wham-george-i-ja',
336:'https://lubimyczytac.pl/ksiazka/4918917/tyle-slonca-anna-jantar-biografia',
337:'https://lubimyczytac.pl/ksiazka/4952225/neurochirurg',
338:'https://lubimyczytac.pl/ksiazka/105075/jedenascie-tysiecy-palek-czyli-milostki-pewnego-hospodara',
339:'https://lubimyczytac.pl/ksiazka/4852217/transformersi-superbohaterowie-polskiej-reklamy-80--90',
340:'https://lubimyczytac.pl/ksiazka/281210/polskie-drogi-w-petersburgu',
341:'https://lubimyczytac.pl/ksiazka/257375/fizjologia-smaku-albo-medytacje-o-gastronomii-doskonalej',
342:'https://lubimyczytac.pl/ksiazka/4224823/imperium',
343:'https://lubimyczytac.pl/ksiazka/4962465/jak-powstala-bomba-atomowa',
344:'https://lubimyczytac.pl/ksiazka/4988781/sapiens-od-zwierzat-do-bogow',
345:'https://lubimyczytac.pl/ksiazka/4988780/21-lekcji-na-xxi-wiek',
346:'https://lubimyczytac.pl/ksiazka/4988782/homo-deus-krotka-historia-jutra',
347:'https://lubimyczytac.pl/ksiazka/3766752/beksinscy-portret-podwojny',
348:'https://lubimyczytac.pl/ksiazka/3764827/tomek-beksinski-portret-prawdziwy',
349:'https://lubimyczytac.pl/ksiazka/3885569/anatomia-milosci-i-zdrady-co-nauka-mowi-o-namietnosciach-czlowieka',
350:'https://lubimyczytac.pl/ksiazka/93282/czlowiek-istota-spoleczna-wybor-tekstow',
351:'https://lubimyczytac.pl/ksiazka/284529/szantaz-emocjonalny-jak-sie-obronic-przed-manipulacja-i-wykorzystaniem',
352:'https://lubimyczytac.pl/ksiazka/306200/meskie-sprawy-zycie-seks-i-cala-reszta',
353:'https://lubimyczytac.pl/ksiazka/4942534/diuna',
354:'https://lubimyczytac.pl/ksiazka/4886323/leonardo-da-vinci',
355:'https://lubimyczytac.pl/ksiazka/4897078/o-polnocy-w-czarnobylu',
356:'https://lubimyczytac.pl/ksiazka/4893591/czarnobyl-historia-nuklearnej-katastrofy',
357:'https://lubimyczytac.pl/ksiazka/282346/kalendarium-lwowa-1918-1939',
358:'https://lubimyczytac.pl/ksiazka/4444475/czlowiek-w-przystepnej-cenie-reportaze-z-tajlandii',
359:'https://lubimyczytac.pl/ksiazka/4807055/nie-hanbi',
360:'https://lubimyczytac.pl/ksiazka/4897409/zrob-sobie-raj',
361:'https://lubimyczytac.pl/ksiazka/4878819/domy-bezdomne',
362:'https://lubimyczytac.pl/ksiazka/4882986/27-smierci-toby-ego-obeda',
363:'https://lubimyczytac.pl/ksiazka/4823321/biale-zwierzeta-sa-bardzo-czesto-gluche',
364:'https://lubimyczytac.pl/ksiazka/4898597/nikt-nie-slucha-reportaze-o-disco-polo',
365:'https://lubimyczytac.pl/ksiazka/4993255/polowanie-na-escobara-historia-najslynniejszego-barona-narkotykowego',
366:'https://lubimyczytac.pl/ksiazka/4931951/co-sie-stalo-w-jonestown-sekta-jima-jonesa-i-najwieksze-zbiorowe-samobojstwo',
367:'https://lubimyczytac.pl/ksiazka/4872701/non-fiction-numer-4-zwierze',
368:'https://lubimyczytac.pl/ksiazka/4881317/non-fiction-numer-5-szkola',
369:'https://lubimyczytac.pl/ksiazka/4902252/non-fiction-numer-6-ziemia',
370:'https://lubimyczytac.pl/ksiazka/4923235/non-fiction-numer-7-media',
371:'https://lubimyczytac.pl/ksiazka/4943094/non-fiction-numer-8-glowa',
372:'https://lubimyczytac.pl/ksiazka/4963541/jezus-niechrystus',
373:'https://lubimyczytac.pl/ksiazka/4971539/kubanczycy-zwykli-ludzie-w-niemozliwym-kraju',
374:'https://lubimyczytac.pl/ksiazka/227917/minimalizm-po-polsku',
375:'https://lubimyczytac.pl/ksiazka/273843/minimalizm-dla-zaawansowanych',
376:'https://lubimyczytac.pl/ksiazka/4955777/zmienic-swiat-raz-jeszcze-jak-wygrac-walke-o-klimat',
377:'https://lubimyczytac.pl/ksiazka/245877/zasada-nieprawdopodobienstwa-dlaczego-codziennie-zdarzaja-sie-cuda-zbiegi-okolicznosci-i-rzadkie-z',
378:'https://lubimyczytac.pl/ksiazka/4856923/wsciekly-kucharz-cala-prawda-o-zdrowym-jedzeniu-i-modnych-dietach',
379:'https://lubimyczytac.pl/ksiazka/171834/sila-nawyku-dlaczego-robimy-to-co-robimy-i-jak-mozna-to-zmienic-w-zyciu-i-biznesie',
380:'https://lubimyczytac.pl/ksiazka/4877804/kwantechizm-czyli-klatka-na-ludzi',
381:'https://lubimyczytac.pl/ksiazka/4915132/statystycznie-rzecz-biorac-czyli-ile-trzeba-zjesc-czekolady-zeby-dostac-nobla',
382:'https://lubimyczytac.pl/ksiazka/4891788/dzieje-lakomstwa-i-obzarstwa',
383:'https://lubimyczytac.pl/ksiazka/216072/cukier-sol-tluszcz-jak-uzalezniaja-nas-koncerny-spozywcze',
384:'https://lubimyczytac.pl/ksiazka/5016930/nie-ma-jednej-rosji',
385:'https://lubimyczytac.pl/ksiazka/4841326/outsiderzy-integracji-europejskiej-szwajcaria-i-norwegia-a-unia-europejska',
386:'https://lubimyczytac.pl/ksiazka/4844019/wielokulturowosc-szwajcarii-na-rozdrozu',
387:'https://lubimyczytac.pl/ksiazka/72388/gazprom-rosyjska-bron',
388:'https://lubimyczytac.pl/ksiazka/295735/sowietstany-podroz-po-turkmenistanie-kazachstanie-tadzykistanie-kirgistanie-i-uzbekistanie',
389:'https://lubimyczytac.pl/ksiazka/4893777/jak-nakarmic-dyktatora',
390:'https://lubimyczytac.pl/ksiazka/4956697/ostatni-bastion-umyslu',
391:'https://lubimyczytac.pl/ksiazka/5019857/rosja-putina',
392:'https://lubimyczytac.pl/ksiazka/213664/strongman-u-szczytu-wladzy',
393:'https://lubimyczytac.pl/ksiazka/240853/oligarchowie-pieniadze-i-wladza-w-kapitalistycznej-rosji',
394:'https://lubimyczytac.pl/ksiazka/4866475/bandera-ikona-putina',
395:'https://lubimyczytac.pl/ksiazka/87710/dmt-molekula-duszy',
396:'https://lubimyczytac.pl/ksiazka/47298/poszukiwanie-przodkow-genealogia-dla-kazdego',
397:'https://lubimyczytac.pl/ksiazka/4898148/renesans-psychodeliczny',
398:'https://lubimyczytac.pl/ksiazka/158490/prawdziwe-halucynacje',
399:'https://lubimyczytac.pl/ksiazka/86690/ayahuasca-swiete-pnacze-duchow',
400:'https://lubimyczytac.pl/ksiazka/182545/teonanacatl-swiete-grzyby',
401:'https://lubimyczytac.pl/ksiazka/4909451/ceremonie-psychodeliczne',
402:'https://lubimyczytac.pl/ksiazka/271440/ropucha-i-jaguar',
403:'https://lubimyczytac.pl/ksiazka/246077/ibogaina-uzdrawiajaca-moc-wizji',
404:'https://lubimyczytac.pl/ksiazka/60429/pokarm-bogow',
405:'https://lubimyczytac.pl/ksiazka/180927/farmakologizacja-wojny-historia-narkotykow-na-polu-bitwy',
406:'https://lubimyczytac.pl/ksiazka/52166/odurzeni-historia-narkotykow-1500---2000',
407:'https://lubimyczytac.pl/ksiazka/308816/lsd-moje-trudne-dziecko-historia-odkrycia-quot-cudownego-narkotyku-quot',
408:'https://lubimyczytac.pl/ksiazka/4887715/czy-psychodeliki-uratuja-swiat',
409:'https://lubimyczytac.pl/ksiazka/105060/doswiadczenie-psychodeliczne',
410:'https://lubimyczytac.pl/ksiazka/63785/odmienny-stan-swiadomosci-historia-kultury-ecstasy-i-acid-house',
411:'https://lubimyczytac.pl/ksiazka/4911409/czarna-owca-medycyny-nieopowiedziana-historia-psychiatrii',
412:'https://lubimyczytac.pl/ksiazka/34178/moj-jezyk-prywatny',
413:'https://lubimyczytac.pl/ksiazka/4848129/pokochawszy-milosc-w-jezyku',
414:'https://lubimyczytac.pl/ksiazka/227746/zapomniane-slowa',
415:'https://lubimyczytac.pl/ksiazka/204306/czerwony-rynek',
416:'https://lubimyczytac.pl/ksiazka/4745023/trupia-farma-sekrety-legendarnego-laboratorium-sadowego',
417:'https://lubimyczytac.pl/ksiazka/4961621/trupia-farma-nowe-sledztwa',
418:'https://lubimyczytac.pl/ksiazka/4898915/co-mowia-zwloki-opowiesci-antropologa-sadowego',
419:'https://lubimyczytac.pl/ksiazka/4947003/co-mowia-kosci-ukryte-tajemnice-zbrodni',
420:'https://lubimyczytac.pl/ksiazka/4892725/krew-pieniadze-medycyna-tajemnice',
421:'https://lubimyczytac.pl/ksiazka/4894178/cialo-nie-klamie',
422:'https://lubimyczytac.pl/ksiazka/4932338/jak-skutecznie-bronic-sie-przed-socjopatami',
423:'https://lubimyczytac.pl/ksiazka/4802406/mindhunter-tajemnice-elitarnej-jednostki-fbi-zajmujacej-sie-sciganiem-seryjnych-przestepcow',
424:'https://lubimyczytac.pl/ksiazka/4848287/profil-mordercy',
425:'https://lubimyczytac.pl/ksiazka/4875299/happy-why-more-or-less-everything-is-absolutely-fine',
426:'https://lubimyczytac.pl/ksiazka/4909259/prostytutki-tajemnice-platnej-milosci',
427:'https://lubimyczytac.pl/ksiazka/4947320/porno-jak-oni-to-robia',
428:'https://lubimyczytac.pl/ksiazka/4956091/bellingcat-ujawniamy-prawde-w-czasach-postprawdy',
429:'https://lubimyczytac.pl/ksiazka/4951581/mila-robotka-polskie-swierszczyki-harlekiny-i-porno-z-satelity',
430:'https://lubimyczytac.pl/ksiazka/4470880/zobaczyc-losia-historia-polskiej-edukacji-seksualnej-od-pierwszej-lekcji-do-internetu',
431:'https://lubimyczytac.pl/ksiazka/236888/prawy-umysl-dlaczego-dobrych-ludzi-dzieli-religia-i-polityka',
432:'https://lubimyczytac.pl/ksiazka/4900718/uduszeni',
433:'https://lubimyczytac.pl/ksiazka/4924236/ksiazka-o-smieciach',
434:'https://lubimyczytac.pl/ksiazka/4860425/mniej-intymny-portret-zakupowy-polakow',
435:'https://lubimyczytac.pl/ksiazka/4917803/turbulencja',
436:'https://lubimyczytac.pl/ksiazka/4985992/czlowiek-o-24-twarzach',
437:'https://lubimyczytac.pl/ksiazka/4895062/sluchajac-psa',
438:'https://lubimyczytac.pl/ksiazka/235693/czarna-ksiega-karmy-dla-zwierzat',
439:'https://lubimyczytac.pl/ksiazka/4934807/dobry-piesek-jak-wychowac-niesfornego-szczeniaka',
440:'https://lubimyczytac.pl/ksiazka/4945370/tajemnica-spotify-muzyczna-rewolucja',
441:'https://lubimyczytac.pl/ksiazka/306485/plutopia-atomowe-miasta-i-nieznane-katastrofy-nuklearne',
442:'https://lubimyczytac.pl/ksiazka/4843448/betonia-dom-dla-kazdego',
443:'https://lubimyczytac.pl/ksiazka/4877381/czarnobyl-instrukcje-przetrwania',
444:'https://lubimyczytac.pl/ksiazka/130712/czarnobylska-modlitwa-kronika-przyszlosci',
445:'https://lubimyczytac.pl/ksiazka/126860/zle-urodzone-reportaze-o-architekturze-prl',
446:'https://lubimyczytac.pl/ksiazka/4867858/nauka-o-klimacie',
447:'https://lubimyczytac.pl/ksiazka/4883679/ilustrowane-prawo-autorskie',
448:'https://lubimyczytac.pl/ksiazka/4821967/krew-cywilizacji-biografia-ropy-naftowej',
449:'https://lubimyczytac.pl/ksiazka/4921407/czarny-labedz-jak-nieprzewidywalne-zdarzenia-rzadza-naszym-zyciem',
450:'https://lubimyczytac.pl/ksiazka/4928183/tesla-geniusz-na-skraju-szalenstwa',
451:'https://lubimyczytac.pl/ksiazka/4889563/a-co-ciebie-obchodzi-co-mysla-inni-dalsze-przypadki-ciekawego-czlowieka',
452:'https://lubimyczytac.pl/ksiazka/3686629/a-slow-death-83-days-of-radiation-sickness',
453:'https://lubimyczytac.pl/ksiazka/4822044/czas-samotnosci-ukraina-i-ukraincy-w-latach-1914-2018',
454:'https://lubimyczytac.pl/ksiazka/271443/wilki',
455:'https://lubimyczytac.pl/ksiazka/4956590/ukrainskie-stulecie-1914-2014-szkice-historyczne',
456:'https://lubimyczytac.pl/ksiazka/4917028/dyktatura-danych',
457:'https://lubimyczytac.pl/ksiazka/4937190/psychiatrzy-sekrety-polskich-gabinetow',
458:'https://lubimyczytac.pl/ksiazka/4939958/mindf-ck-cambridge-analytica-czyli-jak-popsuc-demokracje',
459:'https://lubimyczytac.pl/ksiazka/203589/steve-jobs',
460:'https://lubimyczytac.pl/ksiazka/5021746/narkonomia-jak-prowadzic-kartel-narkotykowy',
461:'https://lubimyczytac.pl/ksiazka/4888861/przewodnik-psychodelicznego-podroznika',
462:'https://lubimyczytac.pl/ksiazka/178469/wyznania-magika',
463:'https://lubimyczytac.pl/ksiazka/154984/jakuck-slownik-miejsca',
464:'https://lubimyczytac.pl/ksiazka/4907715/kochaj-najlepiej-jak-potrafisz-o-sztuce-bycia-razem',
465:'https://lubimyczytac.pl/ksiazka/4890171/wojna-jest-zawsze-przegrana-polscy-reporterzy-wojenni-opowiadaja-o-tym-czego-do-tej-pory-nie',
466:'https://lubimyczytac.pl/ksiazka/4816618/czerwony-glod',
467:'https://lubimyczytac.pl/ksiazka/4978572/jak-ogarnac-swiat-i-odroznic-fakty-od-wyobrazen',
468:'https://lubimyczytac.pl/ksiazka/4964904/klikniesz-kotku-naga-prawda-o-internetowych-pracownicach-seksualnych',
469:'https://lubimyczytac.pl/ksiazka/4891446/na-ciezkim-kacu-nowa-rosja-putina-i-duchy-przeszlosci',
470:'https://lubimyczytac.pl/ksiazka/4944144/to-nie-jest-propaganda-przygody-na-wojnie-z-rzeczywistoscia',
471:'https://lubimyczytac.pl/ksiazka/5002493/ludzie-putina-jak-kgb-odzyskalo-rosje-i-zwrocilo-sie-przeciwko-zachodowi',
472:'https://lubimyczytac.pl/ksiazka/4934814/go-prawda-jak-nie-wpadac-w-pulapki-myslowe',
473:'https://lubimyczytac.pl/ksiazka/4953116/deadline-przezyj-ten-rok-jakby-mial-byc-twoim-ostatnim',
474:'https://lubimyczytac.pl/ksiazka/4347364/on-wrocil',
475:'https://lubimyczytac.pl/ksiazka/4969393/wykresy-fal-srodkowej-wolgi',
476:'https://lubimyczytac.pl/ksiazka/4987142/ksiazka-po-okladce-o-wspolczesnym-polskim-projektowaniu-okladek-ksiazkowych',
477:'https://lubimyczytac.pl/ksiazka/4991563/detal-architektoniczny-warszawskiej-pragi-1850-1939',
478:'https://lubimyczytac.pl/ksiazka/12021/wysadzic-rosje',
479:'https://lubimyczytac.pl/ksiazka/246966/trzecia-wojna-swiatowa-bitwa-o-ukraine',
480:'https://lubimyczytac.pl/ksiazka/267942/co-wy-wiecie-o-rosji',
481:'https://lubimyczytac.pl/ksiazka/4911024/powrot-rosyjskiego-lewiatana',
482:'https://lubimyczytac.pl/ksiazka/4993522/na-kremlu-wiecznie-zima-rosja-za-drugich-rzadow-putina',
483:'https://lubimyczytac.pl/ksiazka/256833/spokojnie-to-tylko-rosja',
484:'https://lubimyczytac.pl/ksiazka/138379/trojka-do-potegi',
485:'https://lubimyczytac.pl/ksiazka/4902875/siostra-anastazja-zycie-pelne-smaku',
486:'https://lubimyczytac.pl/ksiazka/4916870/jezyczni-co-jezyk-robi-naszej-glowie',
487:'https://lubimyczytac.pl/ksiazka/4915353/ciecz-o-cudownych-i-niebezpiecznych-substancjach-ktore-wplywaja-na-nasze-zycie',
488:'https://lubimyczytac.pl/ksiazka/4940059/ciemne-typki-sekretne-zycie-znakow-typograficznych',
489:'https://lubimyczytac.pl/ksiazka/287390/nowy-car-wczesne-lata-i-rzady-wladimira-putina',
490:'https://lubimyczytac.pl/ksiazka/4886309/polakow-seks-powszedni',
491:'https://lubimyczytac.pl/ksiazka/4901108/krecone-siekiera-9-seansow-smarzowskiego',
492:'https://lubimyczytac.pl/ksiazka/4951281/hazardzisci-gra-o-zycie',
493:'https://lubimyczytac.pl/ksiazka/4956992/republika-samsunga-azjatycki-tygrys-ktory-podbil-swiat-technologii',
494:'https://lubimyczytac.pl/ksiazka/4999350/user-friendly-jak-niewidoczne-zasady-projektowania-zmieniaja-nasze-zycie-prace-i-rozrywke',
495:'https://lubimyczytac.pl/ksiazka/4997572/hajland-jak-cpaja-nasze-dzieci',
496:'https://lubimyczytac.pl/ksiazka/104251/pochowek-dla-rezuna',
497:'https://lubimyczytac.pl/ksiazka/4805335/syrop-z-piolunu-wygnani-w-akcji-quot-wisla-quot',
498:'https://lubimyczytac.pl/ksiazka/4911022/moja-osoba-eseje-i-przygody',
499:'https://lubimyczytac.pl/ksiazka/4862324/dlaczego-przegramy-wojne-z-rosja',
500:'https://lubimyczytac.pl/ksiazka/4945452/stewardesy-cala-prawda-o-lataniu',
501:'https://lubimyczytac.pl/ksiazka/4998168/sztuka-kochania-po-wlosku-jak-czerpac-przyjemnosc-z-seksu',
502:'https://lubimyczytac.pl/ksiazka/4998992/nawalny-nemezis-putina-przyszlosc-rosji',
503:'https://lubimyczytac.pl/ksiazka/4862789/male-zycie',
504:'https://lubimyczytac.pl/ksiazka/4981857/antoni-patek-zegarmistrz-krolow-sladami-zycia',
505:'https://lubimyczytac.pl/ksiazka/294703/wniebowziete-o-stewardesach-w-prl-u',
506:'https://lubimyczytac.pl/ksiazka/5001199/sprawy-lozkowe-historia-seksu',
507:'https://lubimyczytac.pl/ksiazka/4930530/chemia-smierci',
508:'https://lubimyczytac.pl/ksiazka/4912455/wolanie-grobu',
509:'https://lubimyczytac.pl/ksiazka/194281/rany-kamieni',
510:'https://lubimyczytac.pl/ksiazka/3932312/szepty-zmarlych',
511:'https://lubimyczytac.pl/ksiazka/309983/zimne-ognie',
512:'https://lubimyczytac.pl/ksiazka/4874534/sztuka-promocji-nauki-praktyczny-poradnik-dla-naukowcow',
513:'https://lubimyczytac.pl/ksiazka/5028785/chirurg',
514:'https://lubimyczytac.pl/ksiazka/168582/kroki',
515:'https://lubimyczytac.pl/ksiazka/5499/miliard-lat-przed-koncem-swiata',
516:'https://lubimyczytac.pl/ksiazka/245866/make-life-harder',
517:'https://lubimyczytac.pl/ksiazka/4022506/make-life-harder-przewodnik-po-polityce-i-nie-tylko-ale-tez',
518:'https://lubimyczytac.pl/ksiazka/4999480/rok-1984',
519:'https://lubimyczytac.pl/ksiazka/4909556/czastki-elementarne',
520:'https://lubimyczytac.pl/ksiazka/237438/przeciez-nie-prosze-o-wiele',
521:'https://lubimyczytac.pl/ksiazka/270394/miasto-szczesliwe-jak-zmienic-nasze-zycie-zmieniajac-nasze-miasta',
522:'https://lubimyczytac.pl/ksiazka/4730090/walka-o-ulice-jak-odzyskac-miasto-dla-ludzi',
523:'https://lubimyczytac.pl/ksiazka/4846184/miasto-na-plus-eseje-o-polskich-przestrzeniach-miejskich',
524:'https://lubimyczytac.pl/ksiazka/4912550/poza-wlasnoscia-w-strone-udanej-polityki-mieszkaniowej',
525:'https://lubimyczytac.pl/ksiazka/4865082/jak-rowery-moga-uratowac-swiat',
526:'https://lubimyczytac.pl/ksiazka/4887390/byc-jak-kopenhaga-dunski-przepis-na-miasto-szczesliwe',
527:'https://lubimyczytac.pl/ksiazka/258794/jako-dowod-i-wyraz-przyjazni-reportaze-o-palacu-kultury',
528:'https://lubimyczytac.pl/ksiazka/252963/miasto-archipelag-polska-mniejszych-miast',
529:'https://lubimyczytac.pl/ksiazka/157099/warszawa-niezaistniala',
530:'https://lubimyczytac.pl/ksiazka/4886434/stacja-plac-dzierzynskiego-czyli-metro-ktorego-warszawa-nie-zobaczyla',
531:'https://lubimyczytac.pl/ksiazka/136858/wanna-z-kolumnada-reportaze-o-polskiej-przestrzeni',
532:'https://lubimyczytac.pl/ksiazka/4897509/sto-lat-warszawskiego-metra-od-pomyslu-do-realizacji',
533:'https://lubimyczytac.pl/ksiazka/296907/ksiega-zachwytow',
534:'https://lubimyczytac.pl/ksiazka/5005445/tako-rzecze-zaratustra',
535:'https://lubimyczytac.pl/ksiazka/4890016/pokolenie-89-mlodzi-o-polskiej-transformacji',
536:'https://lubimyczytac.pl/ksiazka/122398/przewodnik-po-warszawskich-blokowiskach',
537:'https://lubimyczytac.pl/ksiazka/4878593/wystrzegaj-sie-futury',
538:'https://lubimyczytac.pl/ksiazka/297313/beksinski-dzien-po-dniu-konczacego-sie-zycia',
539:'https://lubimyczytac.pl/ksiazka/302729/jak-czytac-obrazy-tresc-forma-technika',
540:'https://lubimyczytac.pl/ksiazka/3766260/piwo-wszystko-co-musisz-wiedziec-zeby-nie-wyjsc-na-glupka',
541:'https://lubimyczytac.pl/ksiazka/4937899/wieden-miasto-najlepsze-do-zycia',
542:'https://lubimyczytac.pl/ksiazka/4918550/szwajcaria-czyli-jak-przezyc-miedzy-krowami-a-bankami-bilet-w-jedna-strone',
543:'https://lubimyczytac.pl/ksiazka/4893389/szwajcaria-i-liechtenstein-alpejska-przygoda-o-smaku-czekolady',
544:'https://lubimyczytac.pl/ksiazka/82804/ck-kuchnia',
545:'https://lubimyczytac.pl/ksiazka/4881518/krotka-historia-jednego-zdjecia',
546:'https://lubimyczytac.pl/ksiazka/4902174/gotowanie-dla-geekow',
547:'https://lubimyczytac.pl/ksiazka/4870561/imperium-big-maca-dzieje-korporacji-mcdonald-s',
548:'https://lubimyczytac.pl/ksiazka/219505/fotografia-zywnosci-od-kuchni',
549:'https://lubimyczytac.pl/ksiazka/265576/monty-python-autobiografia-wedlug-monty-pythona',
550:'https://lubimyczytac.pl/ksiazka/4874523/historia-sztuki',
551:'https://lubimyczytac.pl/ksiazka/251600/palac-w-warszawie',
552:'https://lubimyczytac.pl/ksiazka/160872/swiss-made-jak-funkcjonuje-wielokulturowa-szwajcaria',
553:'https://lubimyczytac.pl/ksiazka/221590/witajcie-w-rosji',
554:'https://lubimyczytac.pl/ksiazka/203962/futu-re',
555:'https://lubimyczytac.pl/ksiazka/4864254/metro-2033',
556:'https://lubimyczytac.pl/ksiazka/4864256/metro-2034',
557:'https://lubimyczytac.pl/ksiazka/4864257/metro-2035',
558:'https://lubimyczytac.pl/ksiazka/4807881/tekst',
559:'https://lubimyczytac.pl/ksiazka/4954845/outpost',
560:'https://lubimyczytac.pl/ksiazka/4988212/outpost-2',
561:'https://lubimyczytac.pl/ksiazka/168658/piknik-na-skraju-drogi',
562:'https://lubimyczytac.pl/ksiazka/4991232/maly-ksiaze',
563:'https://lubimyczytac.pl/ksiazka/4924482/czlowiek-w-poszukiwaniu-sensu',
564:'https://lubimyczytac.pl/ksiazka/87285/wola-sensu-zalozenia-i-zastosowanie-logoterapii',
565:'https://lubimyczytac.pl/ksiazka/137042/bog-ukryty-w-poszukiwaniu-ostatecznego-sensu',
566:'https://lubimyczytac.pl/ksiazka/233311/lekko-stronniczy---jeszcze-wiecej',
567:'https://lubimyczytac.pl/ksiazka/298095/kropki',
568:'https://lubimyczytac.pl/ksiazka/5026087/2001-a-space-odyssey',
569:'https://lubimyczytac.pl/ksiazka/49046/2010-odyseja-kosmiczna',
570:'https://lubimyczytac.pl/ksiazka/248505/cmentarz-lyczakowski-we-lwowie-w-latach-1786-1986',
571:'https://lubimyczytac.pl/ksiazka/154037/biblia-metalowca',
572:'https://lubimyczytac.pl/ksiazka/4876176/szkarlatna-litera',
573:'https://lubimyczytac.pl/ksiazka/47932/ostatni-wyklad',
574:'https://lubimyczytac.pl/ksiazka/4808886/krotka-historia-czasu-od-wielkiego-wybuchu-do-czarnych-dziur',
575:'https://lubimyczytac.pl/ksiazka/4884276/zemsta',
576:'https://lubimyczytac.pl/ksiazka/256770/bedac-kolega-kierownikiem',
577:'https://lubimyczytac.pl/ksiazka/4890495/paragraf-22',
578:'https://lubimyczytac.pl/ksiazka/4646112/ladacznica',
579:'https://lubimyczytac.pl/ksiazka/4987261/ulisses',
580:'https://lubimyczytac.pl/ksiazka/5031141/trzej-muszkieterowie',
581:'https://lubimyczytac.pl/ksiazka/125296/finneganow-tren',
582:'https://lubimyczytac.pl/ksiazka/47009/sztuczki-umyslu',
583:'https://lubimyczytac.pl/ksiazka/47137/mowa-ciala',
584:'https://lubimyczytac.pl/ksiazka/4988989/atlas-osobliwosci',
585:'https://lubimyczytac.pl/ksiazka/123913/taekwondo',
586:'https://lubimyczytac.pl/ksiazka/3976897/wine-folly-the-essential-guide-to-wine',
#587:'https://lubimyczytac.pl/ksiazka/243569/swiatlo-w-fotografii-magia-i-nauka',
588:'https://lubimyczytac.pl/ksiazka/53421/feynmana-wyklady-z-fizyki-t-1-3',
589:'https://lubimyczytac.pl/ksiazka/4912497/salt-fat-acid-heat-cztery-skladniki',
590:'https://lubimyczytac.pl/ksiazka/4848977/soviet-bus-stops',
591:'https://lubimyczytac.pl/ksiazka/4997423/soviet-metro-stations-stancii-sovetskogo-metro',
592:'https://lubimyczytac.pl/ksiazka/3681319/sekrety-twin-peaks',
593:'https://lubimyczytac.pl/ksiazka/4704477/the-final-dossier',
594:'https://lubimyczytac.pl/ksiazka/5023928/folwark-zwierzecy',
595:'https://lubimyczytac.pl/ksiazka/4523241/czy-wiesz-dlaczego-nie-wiesz-kto-jest-prezydentem-szwajcarii',
596:'https://lubimyczytac.pl/ksiazka/310990/harry-potter-i-insygnia-smierci',
597:'https://lubimyczytac.pl/ksiazka/4973920/ojciec-chrzestny',
598:'https://lubimyczytac.pl/ksiazka/5980/nagrody-darwina-ewolucja-w-dzialaniu',
599:'https://lubimyczytac.pl/ksiazka/4491501/sekretny-dziennik-laury-palmer',
600:'https://lubimyczytac.pl/ksiazka/4860910/pan-raczy-zartowac-panie-feynman',
601:'https://lubimyczytac.pl/ksiazka/4866215/rzeznia-numer-piec',
602:'https://lubimyczytac.pl/ksiazka/288482/pokoj',
603:'https://lubimyczytac.pl/ksiazka/153462/nagi-lunch',
604:'https://lubimyczytac.pl/ksiazka/4964197/autostopem-przez-galaktyke',
605:'https://lubimyczytac.pl/ksiazka/38257/fight-club-podziemny-krag',
606:'https://lubimyczytac.pl/ksiazka/4968875/palac-kultury-i-grozy-oraz-inne-warszawskie-mity',
607:'https://lubimyczytac.pl/ksiazka/148067/14-57-do-czyty-reportaze-z-rosji',
608:'https://lubimyczytac.pl/ksiazka/5017808/dziewczyna-z-sasiedztwa',
609:'https://lubimyczytac.pl/ksiazka/4983066/mury-historia-cywilizacji',
610:'https://lubimyczytac.pl/ksiazka/4883107/w-rajskiej-dolinie-wsrod-zielska',
611:'https://lubimyczytac.pl/ksiazka/5031485/dzienniki-kolymskie',
612:'https://lubimyczytac.pl/ksiazka/4879047/biala-goraczka',
613:'https://lubimyczytac.pl/ksiazka/4852071/buszujacy-w-barszczu',
614:'https://lubimyczytac.pl/ksiazka/128425/putin-czlowiek-bez-twarzy',
615:'https://lubimyczytac.pl/ksiazka/55538/moja-europa-dwa-eseje-o-europie-zwanej-srodkowa',
616:'https://lubimyczytac.pl/ksiazka/267945/tatuaz-z-tryzubem',
617:'https://lubimyczytac.pl/ksiazka/4991386/rosja-od-kuchni-jak-zbudowac-imperium-nozem-chochla-i-widelcem',
618:'https://lubimyczytac.pl/ksiazka/4914317/czarne-zloto-wojny-o-wegiel-z-donbasu',
619:'https://lubimyczytac.pl/ksiazka/4980248/zniewolona-rosja-historia-poddanstwa',
620:'https://lubimyczytac.pl/ksiazka/78897/boze-igrzysko-historia-polski',
621:'https://lubimyczytac.pl/ksiazka/6291/akta-litwinienki',
622:'https://lubimyczytac.pl/ksiazka/96392/witajcie-w-raju-reportaze-o-przemysle-turystycznym',
623:'https://lubimyczytac.pl/ksiazka/195468/ukraina-przewodnik-krytyki-politycznej',
624:'https://lubimyczytac.pl/ksiazka/4875709/ja-nie-lecze-ja-uzdrawiam-prawdziwa-twarz-polskich-bioenergoterapeutow',
625:'https://lubimyczytac.pl/ksiazka/63490/putin-i-odrodzenie-rosji',
626:'https://lubimyczytac.pl/ksiazka/4989043/zbrodnia-i-kara',
627:'https://lubimyczytac.pl/ksiazka/4950178/co-robic-przed-koncem-swiata',
628:'https://lubimyczytac.pl/ksiazka/4919383/polacy-last-minute-sekrety-pilotow-wycieczek',
629:'https://lubimyczytac.pl/ksiazka/4936098/byla-zona',
630:'https://lubimyczytac.pl/ksiazka/4960449/zabijaj-uwaznie',
631:'https://lubimyczytac.pl/ksiazka/66860/requiem-dla-snu',
632:'https://lubimyczytac.pl/ksiazka/4878905/zew-cthulhu',
633:'https://lubimyczytac.pl/ksiazka/4994961/armenia-obieg-zamkniety',
634:'https://lubimyczytac.pl/ksiazka/4897785/gareth-jones-czlowiek-ktory-wiedzial-za-duzo',
635:'https://lubimyczytac.pl/ksiazka/4268520/abchazja',
636:'https://lubimyczytac.pl/ksiazka/4972288/brr-jak-zazywac-zimnych-kapieli-aby-wspierac-caly-organizm',
637:'https://lubimyczytac.pl/ksiazka/4893073/zaklamane-jedzenie-obalamy-mity-zdrowego-odzywiania',
638:'https://lubimyczytac.pl/ksiazka/4916282/partnerstwo-bliskosci-jak-teoria-wiezi-pomoze-ci-stworzyc-szczesliwy-zwiazek',
639:'https://lubimyczytac.pl/ksiazka/4996794/jakby-luksusowo-przewodnik-po-architekturze-warszawy-lat-90',
640:'https://lubimyczytac.pl/ksiazka/5013911/wojna-informacyjna',
641:'https://lubimyczytac.pl/ksiazka/148080/psychologia-milosci-intymnosc-namietnosc-zobowiazanie',
642:'https://lubimyczytac.pl/ksiazka/4843823/neuroerotyka-rozmowy-o-seksie-i-nie-tylko',
643:'https://lubimyczytac.pl/ksiazka/4964250/metoda-czarnej-skrzynki-zaskakujaca-prawda-o-nauce-na-bledach',
644:'https://lubimyczytac.pl/ksiazka/212119/sztuka-zycia-wedlug-stoikow',
645:'https://lubimyczytac.pl/ksiazka/50522/jak-zdobyc-przyjaciol-i-zjednac-sobie-ludzi',
646:'https://lubimyczytac.pl/ksiazka/17348/jak-przestac-sie-martwic-i-zaczac-zyc',
647:'https://lubimyczytac.pl/ksiazka/3816755/madrzej-szybciej-lepiej-sekret-efektywnosci',
648:'https://lubimyczytac.pl/ksiazka/4915680/szkoda-ze-twoi-rodzice-nie-przeczytali-tej-ksiazki-a-twoje-dzieci-docenia-ze-ja-znasz',
649:'https://lubimyczytac.pl/ksiazka/170210/nowa-psychologia-sukcesu',
650:'https://lubimyczytac.pl/ksiazka/4367313/wiezniowie-geografii-czyli-wszystko-co-chcialbys-wiedziec-o-globalnej-polityce',
651:'https://lubimyczytac.pl/ksiazka/4980615/potega-geografii-czyli-jak-bedzie-wygladal-w-przyszlosci-nasz-swiat',
652:'https://lubimyczytac.pl/ksiazka/4979572/ekstaza-lata-90-poczatek',
653:'https://lubimyczytac.pl/ksiazka/4881852/granica-podroz-wokol-rosji-przez-koree-polnocna-chiny-mongolie-kazachstan-azerbejdzan-gruzje',
654:'https://lubimyczytac.pl/ksiazka/4800140/jak-zadbac-o-wlasne-finanse',
655:'https://lubimyczytac.pl/ksiazka/4940535/finansowa-forteca',
656:'https://lubimyczytac.pl/ksiazka/314304/finansowy-ninja',
657:'https://lubimyczytac.pl/ksiazka/4993547/narkotyki-bez-paniki-co-trzeba-wiedziec-o-legalnych-i-nielegalnych-srodkach-psychoaktywnych',
658:'https://lubimyczytac.pl/ksiazka/4986936/apartament-w-hotelu-wojna-reportaz-z-donbasu',
659:'https://lubimyczytac.pl/ksiazka/313620/sprawiedliwi-zdrajcy-sasiedzi-z-wolynia',
660:'https://lubimyczytac.pl/ksiazka/4937623/na-du-e-se-zrob',
661:'https://lubimyczytac.pl/ksiazka/3970565/buszujacy-w-zbozu',
662:'https://lubimyczytac.pl/ksiazka/5015741/ksiaze',
663:'https://lubimyczytac.pl/ksiazka/164283/ostatni-swiadkowie-utwory-solowe-na-glos-dzieciecy',
664:'https://lubimyczytac.pl/ksiazka/4889452/opowiedz-mi-o-nim',
665:'https://lubimyczytac.pl/ksiazka/4901443/swiateczny-dyzur',
666:'https://lubimyczytac.pl/ksiazka/4872035/w-czepku-urodzone-o-niewidzialnych-bohaterkach-szpitalnych-korytarzy',
667:'https://lubimyczytac.pl/ksiazka/258953/cynkowi-chlopcy',
668:'https://lubimyczytac.pl/ksiazka/4368657/mlody-stalin',
669:'https://lubimyczytac.pl/ksiazka/4856498/praca-gleboka-jak-odniesc-sukces-w-swiecie-w-ktorym-ciagle-cos-nas-rozprasza',
670:'https://lubimyczytac.pl/ksiazka/4920593/cyfrowy-minimalizm-jak-zachowac-skupienie-w-halasliwym-swiecie',
671:'https://lubimyczytac.pl/ksiazka/4931329/nie-dla-psa-i-kota-kielbasa-czyli-jak-zdrowo-karmic-swojego-zwierzaka',
672:'https://lubimyczytac.pl/ksiazka/4952004/jak-madrze-zadbac-o-swojego-psa-i-kota',
673:'https://lubimyczytac.pl/ksiazka/4939278/erystyka-czyli-sztuka-prowadzenia-sporow',
674:'https://lubimyczytac.pl/ksiazka/4486038/wszyscy-ludzie-kremla-tajne-zycie-dworu-wladimira-putina',
675:'https://lubimyczytac.pl/ksiazka/4872815/droga-do-niewolnosci-rosja-europa-ameryka',
676:'https://lubimyczytac.pl/ksiazka/4943329/wiek-kapitalizmu-inwigilacji',
677:'https://lubimyczytac.pl/ksiazka/4872076/kwestia-rosyjska-jak-budowano-narod-i-imperium',
678:'https://lubimyczytac.pl/ksiazka/4940225/geopolityka-rosji-i-obszaru-postsowieckiego',
679:'https://lubimyczytac.pl/ksiazka/4953660/czy-to-normalne-seks-zwiazki-i-statystyka',
680:'https://lubimyczytac.pl/ksiazka/4903951/sztuka-tworzenia-wspomnien-jak-kreowac-i-zapamietywac-szczesliwe-chwile',
681:'https://lubimyczytac.pl/ksiazka/4973487/pi-razy-oko',
682:'https://lubimyczytac.pl/ksiazka/4910992/krol-darknetu-polowanie-na-genialnego-cyberprzestepce',
683:'https://lubimyczytac.pl/ksiazka/4880213/darknet',
684:'https://lubimyczytac.pl/ksiazka/4868303/atlas-szczescia',
685:'https://lubimyczytac.pl/ksiazka/4970285/biale-platki-zloty-srodek-historie-rodzinne',
686:'https://lubimyczytac.pl/ksiazka/4914197/ostatnie-zlecenie',
687:'https://lubimyczytac.pl/ksiazka/4909368/ewangelia-wedlug-wegorza',
688:'https://lubimyczytac.pl/ksiazka/4992008/filozofia-dla-zabieganych-mala-ksiazeczka-o-wielkich-ideach',
689:'https://lubimyczytac.pl/ksiazka/4951901/szalona-milosc-chce-takiego-jak-putin-reportaze-z-rosji',
690:'https://lubimyczytac.pl/ksiazka/4951277/krwawe-pozdrowienia-z-rosji-jak-wladimir-putin-eliminuje-swoich-przeciwnikow',
691:'https://lubimyczytac.pl/ksiazka/4918551/wschod-is-good',
692:'https://lubimyczytac.pl/ksiazka/4927118/rosja-we-krwi-terroryzm-dwoch-dekad',
693:'https://lubimyczytac.pl/ksiazka/4657542/o-tyranii-dwadziescia-lekcji-z-dwudziestego-wieku',
694:'https://lubimyczytac.pl/ksiazka/4947470/jak-nie-bac-sie-bliskosci-o-budowaniu-dobrych-wiezi',
695:'https://lubimyczytac.pl/ksiazka/5005523/co-ci-sie-przydarzylo',
696:'https://lubimyczytac.pl/ksiazka/4925191/kebabistan',
697:'https://lubimyczytac.pl/ksiazka/4979111/psie-smaki-o-zbilansowanej-diecie-dla-psow',
698:'https://lubimyczytac.pl/ksiazka/4857311/urodzilam-sie-na-grochowie',
699:'https://lubimyczytac.pl/ksiazka/4964140/seksualne-zycie-polakow',
700:'https://lubimyczytac.pl/ksiazka/5008710/wowa-wolodia-wladimir-tajemnice-rosji-putina',
701:'https://lubimyczytac.pl/ksiazka/4876452/kompas-dietetyczny-jak-zdrowo-jesc-i-nie-zwariowac',
702:'https://lubimyczytac.pl/ksiazka/4943448/krol-tygrysow-jest-nagi-trzy-lata-w-egzotycznym-krolestwie-joe-exotica',
703:'https://lubimyczytac.pl/ksiazka/311639/glutenowe-klamstwo-i-inne-mity-o-tym-co-jemy',
704:'https://lubimyczytac.pl/ksiazka/4939197/ludowa-historia-polski',
705:'https://lubimyczytac.pl/ksiazka/4892457/wirus-paniki-historia-kontrowersji-wokol-szczepionek-i-autyzmu',
706:'https://lubimyczytac.pl/ksiazka/4998848/klub-rozkoszy-kartografia-przyjemnosci',
707:'https://lubimyczytac.pl/ksiazka/4893263/pokazucha-na-gruzinskich-zasadach',
708:'https://lubimyczytac.pl/ksiazka/5014098/sto-technik-wplywu-spolecznego-kiedy-i-dlaczego-wywieranie-wplywu-na-innych-jest-skuteczne',
709:'https://lubimyczytac.pl/ksiazka/4974518/kosci-ktore-nosisz-w-kieszeni',
710:'https://lubimyczytac.pl/ksiazka/4935103/po-co-czytac-klasykow',
711:'https://lubimyczytac.pl/ksiazka/4871914/a-co-wyscie-myslaly-spotkania-z-kobietami-z-mazowieckich-wsi',
712:'https://lubimyczytac.pl/ksiazka/4953178/coraz-ciemniej-ekstremisci-w-sieci',
713:'https://lubimyczytac.pl/ksiazka/4871910/farerskie-kadry-wyspy-gdzie-owce-mowia-dobranoc',
714:'https://lubimyczytac.pl/ksiazka/4920179/gniew',
715:'https://lubimyczytac.pl/ksiazka/4535596/halo-tato-reportaze-o-dobrym-ojcostwie',
716:'https://lubimyczytac.pl/ksiazka/4959368/houston-lecimy',
717:'https://lubimyczytac.pl/ksiazka/4884993/made-in-sweden-60-slow-ktore-stworzyly-narod',
718:'https://lubimyczytac.pl/ksiazka/106735/nagrody-darwina-4-inteligentny-projekt',
719:'https://lubimyczytac.pl/ksiazka/4972616/opoznienie-moze-ulec-zmianie',
720:'https://lubimyczytac.pl/ksiazka/4930176/pandemia-covid-19-trzesie-swiatem',
721:'https://lubimyczytac.pl/ksiazka/247214/pozdrowienia-z-noworosji',
722:'https://lubimyczytac.pl/ksiazka/49088/strefa-zdekomunizowana---wywiad-rzeka-z-radoslawem-sikorskim',
723:'https://lubimyczytac.pl/ksiazka/5001815/stulecie-samotnych-jak-odzyskac-utracone-wiezi',
724:'https://lubimyczytac.pl/ksiazka/4926013/w-polsce-czyli-wszedzie-rzecz-o-upadku-i-przyszlosci-swiata',
725:'https://lubimyczytac.pl/ksiazka/167409/w-obronie-kory-i-wolnosci-list-do-premiera',
726:'https://lubimyczytac.pl/ksiazka/4945357/zatyrani-reportaz-o-najgorzej-platnych-pracach',
727:'https://lubimyczytac.pl/ksiazka/4844920/21-polskich-grzechow-glownych',
728:'https://lubimyczytac.pl/ksiazka/299499/berlin-hipsterska-stolica-europy',
729:'https://lubimyczytac.pl/ksiazka/181081/bracia-polscy-przewodnik-krytyki-politycznej',
730:'https://lubimyczytac.pl/ksiazka/54023/bog-zaplac',
731:'https://lubimyczytac.pl/ksiazka/4553284/chcialabym-chciala-o-czym-polki-marza-w-lozku',
732:'https://lubimyczytac.pl/ksiazka/297326/creme-de-la-kreml-172-opowiesci-o-rosji',
733:'https://lubimyczytac.pl/ksiazka/63280/czarodziejki-com',
734:'https://lubimyczytac.pl/ksiazka/4977639/kielbasa-i-sznurek',
735:'https://lubimyczytac.pl/ksiazka/4937897/klub-seksskandal-w-komitecie-noblowskim',
736:'https://lubimyczytac.pl/ksiazka/287886/memy-i-graffy-dzender-kasa-i-seks',
737:'https://lubimyczytac.pl/ksiazka/4137383/miesoholicy-2-5-miliona-lat-miesozerczej-obsesji-czlowieka',
738:'https://lubimyczytac.pl/ksiazka/160090/nielatwy-dzien',
739:'https://lubimyczytac.pl/ksiazka/4917854/odwiedziny-rozmowy-o-dizajnie',
740:'https://lubimyczytac.pl/ksiazka/196392/pozeracze-danych-o-zawlaszczaniu-naszych-danych-i-o-tym-jak-odzyskac-nad-nimi-kontrole',
741:'https://lubimyczytac.pl/ksiazka/4925545/prawo-do-kultury',
742:'https://lubimyczytac.pl/ksiazka/262364/przejezyczenie-rozmowy-o-przekladzie',
743:'https://lubimyczytac.pl/ksiazka/223839/snowden-nigdzie-sie-nie-ukryjesz',
744:'https://lubimyczytac.pl/ksiazka/4938688/szramy-jak-psychosystem-niszczy-nasze-dzieci',
745:'https://lubimyczytac.pl/ksiazka/3970876/toast-za-przodkow',
746:'https://lubimyczytac.pl/ksiazka/254305/ukraina-niezwykli-ludzie-w-niezwyklych-czasach',
747:'https://lubimyczytac.pl/ksiazka/265114/what-if-a-co-gdyby-naukowe-odpowiedzi-na-absurdalne-i-hipotetyczne-pytania',
748:'https://lubimyczytac.pl/ksiazka/151008/zeznania-niekrytego-krytyka',
749:'https://lubimyczytac.pl/ksiazka/156464/wytracony-z-rownowagi',
750:'https://lubimyczytac.pl/ksiazka/4950811/privacy-is-power-why-and-how-you-should-take-back-control-of-your-data'
}

In [ ]:
df

,image,book_title,name,datePublished,numberOfPages,ratingValue,ratingCount,category,publisher,series,readingTime,tags,originalTitle,firstPolishPublish,originPublish,isbn,inLanguage,url
1,https://s.lubimyczytac.pl/upload/books/4949000...,"Zachowuj się. Jak biologia wydobywa z nas to, ...",Robert M. Sapolsky,2021-03-10,1008,"9,1",98,"nauki społeczne (psychologia, socjologia, itd.)",Media Rodzina,NaN,33 h 36 min - 50 h 24 min,"psychologia, zachowuj się, sapolsky, popularno...",Behave: The Biology of Humans at Our Best and ...,2021-03-10,2017-05-02,9788380088276,polski,https://lubimyczytac.pl/ksiazka/4949256/zachow...
2,https://s.lubimyczytac.pl/upload/books/297000/...,Zaraza,Jerzy Ambroziewicz,2016-03-30,280,"7,1",420,reportaż,Dowody na Istnienie,Faktyczny Dom Kultury,9 h 20 min - 14 h 0 min,"ospa, epidemia, reportaż",NaN,1965-01-01,NaN,9788394311841,polski,https://lubimyczytac.pl/ksiazka/297376/zaraza
3,https://s.lubimyczytac.pl/upload/books/4973000...,Gadka. W sześćdziesiąt języków dookoła Europy,Gaston Dorren,2021-08-17,384,"7,3",145,"językoznawstwo, nauka o literaturze",Karakter,NaN,12 h 48 min - 19 h 12 min,"językoznawstwo, lingwistyka",Lingo: Around Europe in Sixty Languages,2021-08-17,2015-01-01,9788366147829,polski,https://lubimyczytac.pl/ksiazka/4973522/gadka-...
4,https://s.lubimyczytac.pl/upload/books/4890000...,Turbopatriotyzm,Marcin Napiórkowski,2019-08-28,320,"7,2",306,"publicystyka literacka, eseje",Czarne,Nowe non-fiction,10 h 40 min - 16 h 0 min,"literatura polska, socjologia, patriotyzm, pol...",NaN,2019-08-28,NaN,9788380499010,polski,https://lubimyczytac.pl/ksiazka/4890328/turbop...
5,https://s.lubimyczytac.pl/upload/books/4889000...,Wędrowny Zakład Fotograficzny,Agnieszka Pajączkowska,2019-08-14,384,"7,9",548,reportaż,Czarne,Poza serią,12 h 48 min - 19 h 12 min,"literatura faktu, niebezpieczeństwo, pogranicz...",NaN,2019-08-14,NaN,9788380498921,polski,https://lubimyczytac.pl/ksiazka/4889761/wedrow...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582,https://s.lubimyczytac.pl/upload/books/47000/4...,Sztuczki umysłu,Derren Brown,2008-01-01,382,"7,2",262,popularnonaukowa,Jacek Santorski & Co,NaN,12 h 44 min - 19 h 6 min,,Tricks of the Mind,2008-01-01,NaN,9788375540772,polski,https://lubimyczytac.pl/ksiazka/47009/sztuczki...
583,https://s.lubimyczytac.pl/upload/books/47000/4...,Mowa ciała,Allan Pease,2008-01-01,456,"7,5",1446,poradniki,Rebis,Poradniki psychologiczne,15 h 12 min - 22 h 48 min,,NaN,2008-01-01,NaN,9788375100198,polski,https://lubimyczytac.pl/ksiazka/47137/mowa-ciala
584,https://s.lubimyczytac.pl/upload/books/4988000...,Atlas osobliwości,Joshua Foer,2021-12-02,472,"7,8",65,literatura podróżnicza,Sonia Draga,NaN,15 h 44 min - 23 h 36 min,"podróż, egzotyka, cuda natury, przewodnik, świ...",Atlas Obscura,2017-11-22,NaN,9788366661264,polski,https://lubimyczytac.pl/ksiazka/4988989/atlas-...
585,https://s.lubimyczytac.pl/upload/books/123000/...,Taekwondo,Lee Myong Kyong,1988-10-01,175,"6,6",14,sport,Alma-Press,NaN,5 h 50 min - 8 h 45 min,"taekwondo, Korea Południowa",NaN,1988-10-01,2008-01-01,83-7020-045-1,polski,https://lubimyczytac.pl/ksiazka/123913/taekwondo


In [ ]:
book_urls = {753:'https://lubimyczytac.pl/ksiazka/141871/oczami-radzieckiej-zabawki'
}

In [ ]:
df_sort_rat = df.sort_values(['ratingCount'],ascending=False)
df_sort_rat.head(50)

,level_0,index,image,book_title,name,datePublished,numberOfPages,ratingValue,ratingCount,category,publisher,series,readingTime,tags,originalTitle,firstPolishPublish,originPublish,isbn,inLanguage,url
265,265,583.0,https://s.lubimyczytac.pl/upload/books/310000/...,Harry Potter i Insygnia Śmierci,J.K. Rowling,2016-07-20,782,"8,4",56227,literatura młodzieżowa,Media Rodzina,NaN,26 h 4 min - 39 h 6 min,"Jonny Duddle, Harry Potter, magia, przygoda",Harry Potter and the Deathly Hallows,2008-01-01,2007-01-01,9788380082236,polski,https://lubimyczytac.pl/ksiazka/310990/harry-p...
474,473,82.0,https://s.lubimyczytac.pl/upload/books/5001000...,Mistrz i Małgorzata,Michaił Bułhakow,2022-03-23,544,"8,4",49822,klasyka,Świat Książki,NaN,18 h 8 min - 27 h 12 min,,Мастер и Маргарита,1969-01-01,2006-02-28,9788381395731,polski,https://lubimyczytac.pl/ksiazka/5001198/mistrz...
225,225,506.0,https://s.lubimyczytac.pl/upload/books/4999000...,Rok 1984,George Orwell,2022-04-05,352,"8,5",44230,"fantasy, science fiction",Rebis,Wehikuł czasu [Rebis],11 h 44 min - 17 h 36 min,,Nineteen eighty-four,1953-01-01,1950-07-01,9788381884334,polski,https://lubimyczytac.pl/ksiazka/4999480/rok-1984
213,213,613.0,https://s.lubimyczytac.pl/upload/books/4989000...,Zbrodnia i kara,Fiodor Dostojewski,2021-10-27,576,"7,9",40649,klasyka,Wydawnictwo MG,NaN,19 h 12 min - 28 h 48 min,"literatura rosyjska, powieść polifoniczna, XIX...",Преступле́ние и наказа́ние,1966-01-01,2000-01-01,9788377797396,polski,https://lubimyczytac.pl/ksiazka/4989043/zbrodn...
514,513,33.0,https://s.lubimyczytac.pl/upload/books/254000/...,Dziewczyna z pociągu,Paula Hawkins,2015-10-21,328,"6,6",35250,"kryminał, sensacja, thriller",Świat Książki,NaN,10 h 56 min - 16 h 24 min,"alkoholizm, tajemnice, pociąg, Jan Kraśko, sus...",The Girl on the Train,2015-10-21,NaN,9788380310650,polski,https://lubimyczytac.pl/ksiazka/254758/dziewcz...
224,224,581.0,https://s.lubimyczytac.pl/upload/books/5023000...,Folwark zwierzęcy,George Orwell,2022-08-23,136,"7,9",30882,"fantasy, science fiction",Rebis,Wehikuł czasu [Rebis],4 h 32 min - 6 h 48 min,,Animal farm,1947-01-01,1951-01-01,9788381885096,polski,https://lubimyczytac.pl/ksiazka/5023928/folwar...
136,136,52.0,https://s.lubimyczytac.pl/upload/books/47000/4...,"My, dzieci z dworca ZOO",Christiane Felscherinow,1987-01-01,244,"7,5",28655,"biografia, autobiografia, pamiętnik",Wydawnictwo Iskry,NaN,8 h 8 min - 12 h 12 min,"narkotyki, literatura młodzieżowa, narkomania,...",Wir Kinder Vom Bahnhof Zoo,1987-01-01,NaN,832070801X,polski,https://lubimyczytac.pl/ksiazka/47662/my-dziec...
264,264,648.0,https://s.lubimyczytac.pl/upload/books/3970000...,Buszujący w zbożu,J.D. Salinger,2016-11-02,304,"6,7",28003,literatura piękna,Albatros,NaN,10 h 8 min - 15 h 12 min,,CATCHER IN THE RYE,1963-01-01,1991-05-01,9788379859115,polski,https://lubimyczytac.pl/ksiazka/3970565/buszuj...
95,95,68.0,https://s.lubimyczytac.pl/upload/books/293000/...,Dzieci z Bullerbyn,Astrid Lindgren,2016-03-16,296,"7,5",27340,literatura dziecięca,Nasza Księgarnia,NaN,9 h 52 min - 14 h 48 min,"Bullerbyn, Lasse, Bosse, Olle, Lisa, Anna, Bri...",Bullerbyboken,1961-01-01,1988-01-01,9788310129659,polski,https://lubimyczytac.pl/ksiazka/293762/dzieci-...
169,169,543.0,https://s.lubimyczytac.pl/upload/books/4864000...,Metro 2033,Dmitry Glukhovsky,2018-10-17,578,"7,8",23828,"fantasy, science fiction",Insignis,Kolekcja 70 lat Empik,19 h 16 min - 28 h 54 min,,Метро 2033,2010-02-24,2005-01-01,9788366071308,polski,https://lubimyczytac.pl/ksiazka/4864254/metro-...
